In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "..", "utilities")))
from format import process_federated_dataset,process_specific_datasets_and_files

# federated_endpoint_sets_to_test = ["Uniprot"]
# # Process the federated dataset and get the output directory
# output_dir = process_federated_dataset(
#     endpoint_sets_to_test=federated_endpoint_sets_to_test
# )

# print(f"Processing complete. Results stored in: {output_dir}")

In [9]:
result_dir = process_specific_datasets_and_files(
    endpoint_files_map={"Uniprot": ["48_glycosylation_sites_and_glycans.ttl", "50_tissues_where_genes_sphingosines_are_expressed.ttl"]}
)

print(f"Processing complete. Results stored in: {result_dir}")

Processing endpoint set: Uniprot

Processed: 50_tissues_where_genes_sphingosines_are_expressed.ttl

Processed: 48_glycosylation_sites_and_glycans.ttl
Processed 2 files from Uniprot
Saved metadata for 2 files from 1 endpoint sets to:
  - /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-07_13-57-51/testset_meta_data.json
Processing complete. Results stored in: /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-07_13-57-51


In [6]:
from datasets import Dataset
from format import load_data_from_file

test_set = load_data_from_file(os.path.join(output_dir, "testset_meta_data.json"))

test_dataset = Dataset.from_dict(test_set)

print(test_dataset[0:1])

{'resource': ['https://sparql.uniprot.org/.well-known/sparql-examples/118_biosodafrontend_rat_brain_human_cancer'], 'natural_language_question': ['What are the Homo sapiens genes associated with cancer and their orthologs expressed in the Rattus norvegicus brain?'], 'query': ['PREFIX up:<http://purl.uniprot.org/core/>\nPREFIX taxon:<http://purl.uniprot.org/taxonomy/>\nPREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>\nPREFIX orth:<http://purl.org/net/orth#>\nPREFIX dcterms:<http://purl.org/dc/terms/>\nPREFIX obo:<http://purl.obolibrary.org/obo/>\nPREFIX lscr:<http://purl.org/lscr#>\nPREFIX genex:<http://purl.org/genex#>\nPREFIX sio: <http://semanticscience.org/resource/>\nSELECT ?gene ?orthologous_protein2 WHERE {\n  {\n    SELECT ?protein1 WHERE {\n      ?protein1 a up:Protein;\n        up:organism/up:scientificName \'Homo sapiens\' ;\n        up:annotation ?annotation .\n      ?annotation rdfs:comment ?annotation_text.\n      ?annotation a up:Disease_Annotation .\n      FILTER CONT

In [ ]:
from langsmith import Client
from datetime import datetime

client = Client()
runs = client.list_runs(project_name="sparql-rag-agent", is_root=True)
first_run = next(runs)


child_runs = list(client.list_runs(project_name="sparql-rag-agent", parent_run_id=first_run.id))


final_state_results = {
    "final_state_response": final_state.get("structured_output", {}).get("query", ""),
    "run_id_langsmith": str(first_run.id),
}

for run in child_runs:
    if run.name == "sparql_query_construction":
        sparql_construction_results = {
            "sparql_construction_prompt_tokens": run.prompt_tokens or 0,
            "sparql_construction_completion_tokens": run.completion_tokens or 0,
            "sparql_construction_total_tokens": run.total_tokens or 0,
            #"sparql_construction_prompt_cost": run.prompt_cost or 0,
            #"sparql_construction_completion_cost": run.completion_cost or 0,
            #"sparql_construction_total_cost": run.total_cost or 0,
        }
    if run.name == "question_understanding":
        question_understanding_results = {
            "question_understanding_prompt_tokens": run.prompt_tokens or 0,
            "question_understanding_completion_tokens": run.completion_tokens or 0,
            "question_understanding_total_tokens": run.total_tokens or 0,
            #"question_understanding_prompt_cost": run.prompt_cost or 0,
            #"question_understanding_completion_cost": run.completion_cost or 0,
            #"question_understanding_total_cost": run.total_cost or 0,
        }


summed_result = {**final_state_results, **sparql_construction_results, **question_understanding_results}

NameError: name 'final_state' is not defined

In [1]:
from langsmith import Client
from datetime import datetime

client = Client()
runs = client.list_runs(project_name="sparql-rag-agent", is_root=True)
first_run = next(runs)


child_runs = list(client.list_runs(project_name="sparql-rag-agent", parent_run_id=first_run.id))

#print(child_runs)

# Filter to find the sparql_query_construction run
sparql_construction_run = None
for run in child_runs:
    if run.name == "sparql_query_construction":
        print(run)
        break



id=UUID('9e7cbeb9-4484-4104-8973-417c928b1960') name='sparql_query_construction' start_time=datetime.datetime(2025, 4, 16, 18, 35, 48, 891629) run_type='chain' end_time=datetime.datetime(2025, 4, 16, 18, 35, 49, 30211) extra={'inputs_is_truthy': True, 'metadata': {'LANGSMITH_ENDPOINT': 'https://eu.api.smith.langchain.com', 'LANGSMITH_PROJECT': 'sparql-rag-agent', 'LANGSMITH_TRACING': 'true', 'langgraph_checkpoint_ns': 'sparql_query_construction:846329f9-e462-836e-4178-60bc85b4cd88', 'langgraph_node': 'sparql_query_construction', 'langgraph_path': ['__pregel_pull', 'sparql_query_construction'], 'langgraph_step': 3, 'langgraph_triggers': ['branch:to:sparql_query_construction'], 'revision_id': 'f15b593-dirty', 'ls_run_depth': 1}, 'runtime': {'langchain_core_version': '0.3.50', 'langchain_version': '0.3.22', 'library': 'langsmith', 'platform': 'macOS-14.4.1-arm64-arm-64bit', 'py_implementation': 'CPython', 'runtime': 'python', 'runtime_version': '3.11.5', 'sdk': 'langsmith-py', 'sdk_versio

In [1]:
import asyncio
import sys
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.evaluation import AgentEvaluator


evaluator = AgentEvaluator(
    #endpoint_sets={"Uniprot": ["43_patents_used_in_uniprot_granted_twenty_years_in_the_past.ttl"]},
    endpoint_sets={"SwissLipids": ["34.ttl"]},
    test=True,
    experiment_dir="eval",
    timeout=300,
)

await evaluator.run_all_tests()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sebastian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Processing endpoint set: SwissLipids
Processed: 34.ttl
Processed 1 files from SwissLipids
Saved metadata for 1 files from 1 endpoint sets to:
  - /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-21_15-06-41/testset_meta_data.json
Sending question 1/1 to agent...


I0000 00:00:1745240803.576556  183869 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Got result for question 1/1
Querying ground truth endpoint...
  Using cached result for query
Querying predicted endpoint...


/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/.venv/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


SP-BLEU 1.0727008290112629e-78
METEOR 0.16943670382165604
F1-score:  0
SP-F1-score:  0
Number of invalid queries(None): 0
Saved query comparison to: /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-21_15-06-41/34_comparison.ttl


'/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-21_15-06-41/evaluation_dataset.json'

# Metric testing

In [3]:
# Create test pairs for eval_pairs function with comments to test clean_sparql_query
test_pairs = [
    (
        """# This is a comment at the beginning of the query
PREFIX rh: <[http://rdf.rhea-db.org/>](http://rdf.rhea-db.org/>)
PREFIX up: <[http://purl.uniprot.org/core/>](http://purl.uniprot.org/core/>) # Comment after a PREFIX declaration

# Query 12 - This is a descriptive comment
# Select all Rhea reactions used to annotate enzyme sequences in UniProtKB
SELECT ?uniprotCount ?rhea ?accession ?equation # Comment after SELECT
WHERE {
  SERVICE <[https://sparql.uniprot.org/sparql>](https://sparql.uniprot.org/sparql>) { # Comment inside the query
    SELECT ?rhea (count(?uniprot) as ?uniprotCount) {
      ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea . # Technical comment
    }
    GROUP BY ?rhea # Grouping comment
  }
  ?rhea rh:accession ?accession . # Comment with special characters: @#$%^&*()
  ?rhea rh:equation ?equation .
} # Final comment at the end of the query""",
        
        """# [https://sparql.rhea-db.org/sparql](https://sparql.rhea-db.org/sparql)
PREFIX rhea: <[http://rdf.rhea-db.org/>](http://rdf.rhea-db.org/>) # Namespace for Rhea

# This query selects all reactions
# It's a simplified version of the ground truth query
SELECT DISTINCT ?reaction # Get unique reactions only
WHERE {
  ?reaction a rhea:Reaction . # This finds all instances of the Reaction class
  # The following line is commented out but would filter by status
  # ?reaction rhea:status "approved" .
}"""
    ),
    
    # Second example with a different query pair and different comment styles
    (
        """PREFIX CHEBI: <[http://purl.obolibrary.org/obo/CHEBI_>](http://purl.obolibrary.org/obo/CHEBI_>) # Chemical Entities of Biological Interest
PREFIX rh: <[http://rdf.rhea-db.org/>](http://rdf.rhea-db.org/>)
PREFIX skos: <[http://www.w3.org/2004/02/skos/core#>](http://www.w3.org/2004/02/skos/core#>)
PREFIX up: <[http://purl.uniprot.org/core/>](http://purl.uniprot.org/core/>)

# This query finds proteins linked to arachidonate
# CHEBI:32395 is the identifier for arachidonate
SELECT 
    DISTINCT # Ensure unique results
        ?uniprot
        ?uniprotID
        ?recname
        ?gene
        ?chebi
        ?uniprotName # Name from UniProt
WHERE {
  SERVICE <[https://sparql.rhea-db.org/sparql>](https://sparql.rhea-db.org/sparql>) { # Federated query to Rhea
     VALUES (?chebi) { (CHEBI:32395) } # Specify the chemical entity
     ?rhea rh:side/rh:contains/rh:compound ?compound . # Path to compound
     ?compound rh:chebi ?chebi .
     ?chebi up:name ?uniprotName . # Get the name
  }
  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea . # Link to reaction
  ?uniprot up:mnemonic ?uniprotID . # Get the ID
  ?uniprot up:recommendedName/up:fullName ?recname .
  OPTIONAL {?uniprot up:encodedBy/skos:prefLabel ?gene .} # Gene name is optional
} # End of query""",
        
        """# This is the predicted query for arachidonate proteins
# It uses different prefixes than the ground truth
PREFIX obo: <[http://purl.obolibrary.org/obo/>](http://purl.obolibrary.org/obo/>) # Ontology prefix
PREFIX rhea: <[http://rdf.rhea-db.org/reaction/>](http://rdf.rhea-db.org/reaction/>) # Different Rhea prefix

SELECT DISTINCT ?protein ?reaction ?ecnumber # Different variables
WHERE {
    # Identify the CHEBI ID for arachidonate
    VALUES ?chebi {obo:CHEBI_32395} # Same entity, different prefix format
    
    # Query Rhea to find reactions associated with arachidonate
    SERVICE <[https://sparql.rhea-db.org/sparql>](https://sparql.rhea-db.org/sparql>) { # Same service endpoint
        ?reaction rhea:side ?side . # Different property path
        ?side rhea:contains ?chebi .
        ?reaction rhea:catalyzedBy ?protein . # Different relationship
    }
    
    # Retrieve EC numbers associated with the reactions
    ?reaction obo:RO_0000057 ?ecnumber . # Additional information
    # ?protein rdfs:label ?name . # This line is commented out
}"""
    )
]

# Example usage in your notebook:
# import sys
# sys.path.append('../utilities')
# from metrics import eval_pairs
# from format import clean_sparql_query
# 
# # Clean the queries before evaluation
# cleaned_pairs = [(clean_sparql_query(gt), clean_sparql_query(pred)) for gt, pred in test_pairs]
# 
# # Run evaluation on cleaned pairs
# results = eval_pairs(cleaned_pairs)
# print(results)

In [2]:
import sys,os


sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')
#from metrics import eval_pairs
from experiments.utilities.format import clean_sparql_query

# Clean the queries before evaluation
cleaned_pairs = [(clean_sparql_query(gt), clean_sparql_query(pred)) for gt, pred in test_pairs]

print(test_pairs)
#print(cleaned_pairs)

[('# This is a comment at the beginning of the query\nPREFIX rh: <[http://rdf.rhea-db.org/>](http://rdf.rhea-db.org/>)\nPREFIX up: <[http://purl.uniprot.org/core/>](http://purl.uniprot.org/core/>) # Comment after a PREFIX declaration\n\n# Query 12 - This is a descriptive comment\n# Select all Rhea reactions used to annotate enzyme sequences in UniProtKB\nSELECT ?uniprotCount ?rhea ?accession ?equation # Comment after SELECT\nWHERE {\n  SERVICE <[https://sparql.uniprot.org/sparql>](https://sparql.uniprot.org/sparql>) { # Comment inside the query\n    SELECT ?rhea (count(?uniprot) as ?uniprotCount) {\n      ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea . # Technical comment\n    }\n    GROUP BY ?rhea # Grouping comment\n  }\n  ?rhea rh:accession ?accession . # Comment with special characters: @#$%^&*()\n  ?rhea rh:equation ?equation .\n} # Final comment at the end of the query', '# [https://sparql.rhea-db.org/sparql](https://sparql.rhea-db.org/sparql)\nPREFIX rhe

In [4]:
import sys,os

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.metrics import eval_pairs
from experiments.utilities.format import clean_sparql_query

cleaned_pairs = [(clean_sparql_query(gt), clean_sparql_query(pred)) for gt, pred in test_pairs]

eval = eval_pairs(cleaned_pairs)

print(eval)

/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/.venv/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/.venv/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


SP-BLEU 9.266129794683117e-79
METEOR 0.19198805586465886
F1-score:  0
SP-F1-score:  0
Number of invalid queries(None): 0
{'SP-BLEU': 9.266129794683117e-79, 'METEOR': 0.19198805586465886, 'f1': 0, 'sp_f1': 0, 'num_none_queries': 0}


In [7]:
identical_query = """# This is a SPARQL query with comments
PREFIX rdf: <[http://www.w3.org/1999/02/22-rdf-syntax-ns#>](http://www.w3.org/1999/02/22-rdf-syntax-ns#>)
PREFIX rdfs: <[http://www.w3.org/2000/01/rdf-schema#>](http://www.w3.org/2000/01/rdf-schema#>) # RDF Schema namespace

# This query finds all classes in the dataset
SELECT DISTINCT ?class ?label # Get classes and their labels
WHERE {
  ?instance a ?class . # This finds all classes used in the dataset
  OPTIONAL { 
    ?class rdfs:label ?label . # Get the label if available
    # Labels make the results more readable
  }
} 
ORDER BY ?class # Sort alphabetically by class URI
LIMIT 100 # Only return the first 100 results
# End of query"""

test_pairs = [
    (identical_query, identical_query),  # Same query for ground truth and prediction
    (identical_query, identical_query)   # Second pair, also identical
]

In [ ]:
#!/usr/bin/env python3
import sys
import os
import json
from datetime import datetime

# Add project root to path
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.metrics import eval_pairs
from experiments.utilities.format import clean_sparql_query

# Assuming test_pairs is defined in the environment where this script is run
# Clean the SPARQL queries
cleaned_pairs = [(clean_sparql_query(gt), clean_sparql_query(pred)) for gt, pred in test_pairs]

# Run evaluation
evaluation_results = eval_pairs(cleaned_pairs)

# Create output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"evaluation_results_{timestamp}.json"

# Store both original and cleaned queries
pairs_data = []
for i, ((original_gt, original_pred), (cleaned_gt, cleaned_pred)) in enumerate(zip(test_pairs, cleaned_pairs)):
    pairs_data.append({
        "pair_id": i,
        "original_ground_truth": original_gt,
        "original_prediction": original_pred,
        "cleaned_ground_truth": cleaned_gt,
        "cleaned_prediction": cleaned_pred
    })


results_dict = {
    "metrics": {metric: value for metric, value in evaluation_results.items()},
    "num_pairs_evaluated": len(test_pairs),
    "query_pairs": pairs_data
}

# Write to JSON file
with open(output_path, 'w') as f:
    json.dump(results_dict, f, indent=2)

print(f"Evaluation results saved to: {output_path}")
print(f"Metrics: {evaluation_results}")

SP-BLEU 1.0
METEOR 0.9999794989544467
F1-score:  0
SP-F1-score:  0
Number of invalid queries(None): 0
Evaluation results saved to: evaluation_results_20250412_162049.json
Metrics: {'SP-BLEU': 1.0, 'METEOR': 0.9999794989544467, 'f1': 0, 'sp_f1': 0, 'num_none_queries': 0}


# Result comparision development

In [ ]:
import sys
import os

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from entity_indexing.endpoint_loader import query_sparql

In [4]:
ground_truth_query = """PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX rh: <http://rdf.rhea-db.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX up: <http://purl.uniprot.org/core/>


SELECT 
    DISTINCT
        ?uniprot
        ?uniprotID
        ?recname
        ?gene
        ?chebi
        ?uniprotName
WHERE {
  SERVICE <https://sparql.rhea-db.org/sparql> {
     VALUES (?chebi) { (CHEBI:32395) }
     ?rhea rh:side/rh:contains/rh:compound ?compound .
     ?compound rh:chebi ?chebi .
     ?chebi up:name ?uniprotName .
  }
  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .
  ?uniprot up:mnemonic ?uniprotID .
  ?uniprot up:recommendedName/up:fullName ?recname .
  OPTIONAL {?uniprot up:encodedBy/skos:prefLabel ?gene .}
}
LIMIT 100
"""

ground_truth_endpoint = "https://sparql.uniprot.org/sparql/"

In [5]:
predicted_query = """PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX rhea: <http://rdf.rhea-db.org/reaction/>

SELECT DISTINCT ?protein ?reaction ?ecnumber
WHERE {
    # Identify the CHEBI ID for arachidonate
    VALUES ?chebi {obo:CHEBI_32395}
    
    # Query Rhea to find reactions associated with arachidonate
    SERVICE <https://sparql.rhea-db.org/sparql> {
        ?reaction rhea:side ?side .
        ?side rhea:contains ?chebi .
        ?reaction rhea:catalyzedBy ?protein .
    }
    
    # Retrieve EC numbers associated with the reactions
    ?reaction obo:RO_0000057 ?ecnumber .
}"""

predicted_endpoint = "https://sparql.uniprot.org/sparql/"


In [6]:
bindings_ground_truth = query_sparql(ground_truth_query, ground_truth_endpoint)["results"]["bindings"]
print(bindings_ground_truth)

[{'uniprotName': {'type': 'literal', 'value': '(5Z,8Z,11Z,14Z)-eicosatetraenoate'}, 'chebi': {'type': 'uri', 'value': 'http://purl.obolibrary.org/obo/CHEBI_32395'}, 'gene': {'type': 'literal', 'value': 'CYP1A2'}, 'uniprot': {'type': 'uri', 'value': 'http://purl.uniprot.org/uniprot/O77809'}, 'uniprotID': {'type': 'literal', 'value': 'CP1A2_MACFA'}, 'recname': {'type': 'literal', 'value': 'Cytochrome P450 1A2'}}, {'uniprotName': {'type': 'literal', 'value': '(5Z,8Z,11Z,14Z)-eicosatetraenoate'}, 'chebi': {'type': 'uri', 'value': 'http://purl.obolibrary.org/obo/CHEBI_32395'}, 'gene': {'type': 'literal', 'value': 'CYP1A2'}, 'uniprot': {'type': 'uri', 'value': 'http://purl.uniprot.org/uniprot/O77810'}, 'uniprotID': {'type': 'literal', 'value': 'CP1A2_CALJA'}, 'recname': {'type': 'literal', 'value': 'Cytochrome P450 1A2'}}, {'uniprotName': {'type': 'literal', 'value': '(5Z,8Z,11Z,14Z)-eicosatetraenoate'}, 'chebi': {'type': 'uri', 'value': 'http://purl.obolibrary.org/obo/CHEBI_32395'}, 'gene':

In [13]:
results_predicted = query_sparql(predicted_query, target_endpoint)
print(results_predicted)

error


In [9]:
import pandas as pd

df = pd.DataFrame(results_ground_truth)

display(df.head())

,uniprotName,chebi,gene,uniprot,uniprotID,recname
0,"{'type': 'literal', 'value': '(5Z,8Z,11Z,14Z)-...","{'type': 'uri', 'value': 'http://purl.obolibra...","{'type': 'literal', 'value': 'CYP1A2'}","{'type': 'uri', 'value': 'http://purl.uniprot....","{'type': 'literal', 'value': 'CP1A2_MACFA'}","{'type': 'literal', 'value': 'Cytochrome P450 ..."
1,"{'type': 'literal', 'value': '(5Z,8Z,11Z,14Z)-...","{'type': 'uri', 'value': 'http://purl.obolibra...","{'type': 'literal', 'value': 'CYP1A2'}","{'type': 'uri', 'value': 'http://purl.uniprot....","{'type': 'literal', 'value': 'CP1A2_CALJA'}","{'type': 'literal', 'value': 'Cytochrome P450 ..."
2,"{'type': 'literal', 'value': '(5Z,8Z,11Z,14Z)-...","{'type': 'uri', 'value': 'http://purl.obolibra...","{'type': 'literal', 'value': 'Cyp1a1'}","{'type': 'uri', 'value': 'http://purl.uniprot....","{'type': 'literal', 'value': 'CP1A1_MOUSE'}","{'type': 'literal', 'value': 'Cytochrome P450 ..."
3,"{'type': 'literal', 'value': '(5Z,8Z,11Z,14Z)-...","{'type': 'uri', 'value': 'http://purl.obolibra...","{'type': 'literal', 'value': 'Cyp1a2'}","{'type': 'uri', 'value': 'http://purl.uniprot....","{'type': 'literal', 'value': 'CP1A2_MOUSE'}","{'type': 'literal', 'value': 'Cytochrome P450 ..."
4,"{'type': 'literal', 'value': '(5Z,8Z,11Z,14Z)-...","{'type': 'uri', 'value': 'http://purl.obolibra...","{'type': 'literal', 'value': 'CYP1A2'}","{'type': 'uri', 'value': 'http://purl.uniprot....","{'type': 'literal', 'value': 'CP1A2_RABIT'}","{'type': 'literal', 'value': 'Cytochrome P450 ..."


In [15]:
extracted_data = []

for row_binding in bindings_ground_truth:
    processed_row = {}
    for var_name, value_dict in row_binding.items():
        if isinstance(value_dict, dict) and 'value' in value_dict:
            processed_row[var_name] = value_dict['value']
        else:
            processed_row[var_name] = None
    extracted_data.append(processed_row)

print(extracted_data)

[{'uniprotName': '(5Z,8Z,11Z,14Z)-eicosatetraenoate', 'chebi': 'http://purl.obolibrary.org/obo/CHEBI_32395', 'gene': 'CYP1A2', 'uniprot': 'http://purl.uniprot.org/uniprot/O77809', 'uniprotID': 'CP1A2_MACFA', 'recname': 'Cytochrome P450 1A2'}, {'uniprotName': '(5Z,8Z,11Z,14Z)-eicosatetraenoate', 'chebi': 'http://purl.obolibrary.org/obo/CHEBI_32395', 'gene': 'CYP1A2', 'uniprot': 'http://purl.uniprot.org/uniprot/O77810', 'uniprotID': 'CP1A2_CALJA', 'recname': 'Cytochrome P450 1A2'}, {'uniprotName': '(5Z,8Z,11Z,14Z)-eicosatetraenoate', 'chebi': 'http://purl.obolibrary.org/obo/CHEBI_32395', 'gene': 'Cyp1a1', 'uniprot': 'http://purl.uniprot.org/uniprot/P00184', 'uniprotID': 'CP1A1_MOUSE', 'recname': 'Cytochrome P450 1A1'}, {'uniprotName': '(5Z,8Z,11Z,14Z)-eicosatetraenoate', 'chebi': 'http://purl.obolibrary.org/obo/CHEBI_32395', 'gene': 'Cyp1a2', 'uniprot': 'http://purl.uniprot.org/uniprot/P00186', 'uniprotID': 'CP1A2_MOUSE', 'recname': 'Cytochrome P450 1A2'}, {'uniprotName': '(5Z,8Z,11Z,14Z

In [16]:
df = pd.DataFrame(extracted_data)

display(df.head())

,uniprotName,chebi,gene,uniprot,uniprotID,recname
0,"(5Z,8Z,11Z,14Z)-eicosatetraenoate",http://purl.obolibrary.org/obo/CHEBI_32395,CYP1A2,http://purl.uniprot.org/uniprot/O77809,CP1A2_MACFA,Cytochrome P450 1A2
1,"(5Z,8Z,11Z,14Z)-eicosatetraenoate",http://purl.obolibrary.org/obo/CHEBI_32395,CYP1A2,http://purl.uniprot.org/uniprot/O77810,CP1A2_CALJA,Cytochrome P450 1A2
2,"(5Z,8Z,11Z,14Z)-eicosatetraenoate",http://purl.obolibrary.org/obo/CHEBI_32395,Cyp1a1,http://purl.uniprot.org/uniprot/P00184,CP1A1_MOUSE,Cytochrome P450 1A1
3,"(5Z,8Z,11Z,14Z)-eicosatetraenoate",http://purl.obolibrary.org/obo/CHEBI_32395,Cyp1a2,http://purl.uniprot.org/uniprot/P00186,CP1A2_MOUSE,Cytochrome P450 1A2
4,"(5Z,8Z,11Z,14Z)-eicosatetraenoate",http://purl.obolibrary.org/obo/CHEBI_32395,CYP1A2,http://purl.uniprot.org/uniprot/P00187,CP1A2_RABIT,Cytochrome P450 1A2


In [ ]:
def result_score(ground_truth_query, ground_truth_endpoint, predicted_query, predicted_endpoint):
    
    ground_truth_data, predicted_data = format_query_result_dataframe(ground_truth_query, ground_truth_endpoint, predicted_query, predicted_endpoint)
    
    metrics = {
        ground_t
    }



In [ ]:
import pandas as pd

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.query_cache import cached_query_sparql

def format_query_result_dataframe(ground_truth_query, ground_truth_endpoint, predicted_query, predicted_endpoint) -> (pd.DataFrame, pd.DataFrame):


    ground_truth = cached_query_sparql(query = ground_truth_query, endpoint_url = ground_truth_endpoint)
    predicted = query_sparql(predicted_query, predicted_endpoint)

    if ground_truth == 'error':

        df_ground_truth = pd.DataFrame()
    else:
        processed_data = []
        bindings_ground_truth = ground_truth['results']['bindings']
        for row_binding in bindings_ground_truth:
            processed_row = {}
            for var_name, value_dict in row_binding.items():
                if isinstance(value_dict, dict) and 'value' in value_dict:
                    processed_row[var_name] = value_dict['value']
                else:
                    processed_row[var_name] = None
            processed_data.append(processed_row)
        df_ground_truth = pd.DataFrame(processed_data)
    
    if predicted == 'error':
        df_predicted = pd.DataFrame()
    else:
        processed_data = []
        bindings_predicted = predicted['results']['bindings']
        for row_binding in bindings_predicted:
            processed_row = {}
            for var_name, value_dict in row_binding.items():
                if isinstance(value_dict, dict) and 'value' in value_dict:
                    processed_row[var_name] = value_dict['value']
                else:
                    processed_row[var_name] = None
            processed_data.append(processed_row)
        df_predicted = pd.DataFrame(processed_data)
        
    return df_ground_truth, df_predicted
    
    

In [23]:
ground_truth_data, predicted_data = format_query_result_dataframe(ground_truth_query, ground_truth_endpoint, predicted_query, predicted_endpoint)

In [33]:
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.format import process_specific_datasets_and_files

In [35]:
directory = process_specific_datasets_and_files(endpoint_files_map={"Uniprot": [], "Rhea": [], "SwissLipids": []}, output_dir="endpoint_parse_testing_ground_truth")

Processing endpoint set: Uniprot

Processed: 118_biosodafrontend_rat_brain_human_cancer.ttl

Processed: 50_tissues_where_genes_sphingosines_are_expressed.ttl

Processed: 51_all_proteins_linked_to_arachidonate.ttl

Processed: 46_duck_pictures_via_eepa_and_taxonomy.ttl

Processed: 40_human_enzymes_that_metabolize_sphingolipids.ttl

Processed: 92_uniprot_bioregistry_iri_translation.ttl

Processed: 49_tissues_where_genes_metabolizing_cholestrol_are_expressed.ttl

Processed: 53_mouse_homologs_of_sterol_enzymes_via_omabrowser.ttl

Processed: 52_drugs_targeting_sterol_metabolism.ttl

Processed: 109_uniprot_transporter_in_liver.ttl

Processed: 54_binding_sites_for_ligands_similar_to_heme_sachem.ttl

Processed: 43_patents_used_in_uniprot_granted_twenty_years_in_the_past.ttl

Processed: 90_uniprot_affected_by_metabolic_diseases_using_MeSH.ttl

Processed: 45_drugs_targeting_human_sterol_metabolism_enzymes.ttl

Processed: 48_glycosylation_sites_and_glycans.ttl

Processed: 42_connect_patents_to_epo

In [1]:


test_set = [
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/118_biosodafrontend_rat_brain_human_cancer",
    "natural_language_question": "What are the Homo sapiens genes associated with cancer and their orthologs expressed in the Rattus norvegicus brain?",
    "query": "PREFIX up:<http://purl.uniprot.org/core/>\nPREFIX taxon:<http://purl.uniprot.org/taxonomy/>\nPREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>\nPREFIX orth:<http://purl.org/net/orth#>\nPREFIX dcterms:<http://purl.org/dc/terms/>\nPREFIX obo:<http://purl.obolibrary.org/obo/>\nPREFIX lscr:<http://purl.org/lscr#>\nPREFIX genex:<http://purl.org/genex#>\nPREFIX sio: <http://semanticscience.org/resource/>\nSELECT ?gene ?orthologous_protein2 WHERE {\n  {\n    SELECT ?protein1 WHERE {\n      ?protein1 a up:Protein;\n        up:organism/up:scientificName 'Homo sapiens' ;\n        up:annotation ?annotation .\n      ?annotation rdfs:comment ?annotation_text.\n      ?annotation a up:Disease_Annotation .\n      FILTER CONTAINS (?annotation_text, \"cancer\")\n    }\n  }\n  SERVICE <https://sparql.omabrowser.org/sparql/> {\n    SELECT ?orthologous_protein2 ?protein1 ?gene WHERE {\n      ?protein_OMA a orth:Protein .\n      ?orthologous_protein2 a orth:Protein .\n      ?cluster a orth:OrthologsCluster .\n      ?cluster orth:hasHomologousMember ?node1 .\n      ?cluster orth:hasHomologousMember ?node2 .\n      ?node2 orth:hasHomologousMember* ?protein_OMA .\n      ?node1 orth:hasHomologousMember* ?orthologous_protein2 \n      .?orthologous_protein2 orth:organism/obo:RO_0002162/up:scientificName 'Rattus norvegicus' .\n      ?orthologous_protein2 sio:SIO_010079 ?gene .\n      ?protein_OMA lscr:xrefUniprot ?protein1 .\n      FILTER(?node1 != ?node2)\n    }\n  }\n  SERVICE <https://www.bgee.org/sparql/> {\n    ?gene genex:isExpressedIn ?anatEntity .\n    ?anatEntity rdfs:label 'brain' .\n    ?gene orth:organism ?org . \n    ?org obo:RO_0002162 taxon:10116 .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql",
    "federates_with": "https://sparql.omabrowser.org/sparql/",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/118_biosodafrontend_rat_brain_human_cancer.ttl",
    "filename": "118_biosodafrontend_rat_brain_human_cancer.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/50",
    "natural_language_question": "Where are the human genes encoding enzymes metabolizing N-acyl\nsphingosines expressed in the human body (federated query, with Rhea and Bgee)",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX genex: <http://purl.org/genex#>\nPREFIX lscr: <http://purl.org/lscr#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  DISTINCT\n    ?chebi\n    ?reaction\n    ?protein ?ensemblGene\n    ?anatomicEntityLabel\n    ?anatomicEntity\nWHERE {\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?reaction rdfs:subClassOf rh:Reaction .\n    ?reaction rh:equation ?reactionEquation .\n    ?reaction rh:side ?reactionSide .\n    ?reactionSide rh:contains ?participant .\n    ?participant rh:compound ?compound .\n    ?compound rh:chebi ?chebi .\n    ?chebi rdfs:subClassOf* CHEBI:52639\n}\n  ?protein up:organism taxon:9606 .\n  ?protein up:annotation ?a .\n  ?a a up:Catalytic_Activity_Annotation .\n  ?a up:catalyticActivity ?ca .\n  ?ca up:catalyzedReaction ?reaction .\n  ?protein rdfs:seeAlso / up:transcribedFrom ?ensemblGene .\n\n  SERVICE  <https://www.bgee.org/sparql/> {\n    ?gene genex:isExpressedIn ?anatomicEntity .\n    ?gene lscr:xrefEnsemblGene ?ensemblGene .\n    ?anatomicEntity rdfs:label ?anatomicEntityLabel .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/50_tissues_where_genes_sphingosines_are_expressed.ttl",
    "filename": "50_tissues_where_genes_sphingosines_are_expressed.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/51",
    "natural_language_question": "Find all proteins linked to arachidonate (CHEBI:32395)",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\n\nSELECT \n    DISTINCT\n        ?uniprot\n        ?uniprotID\n        ?recname\n        ?gene\n        ?chebi\n        ?uniprotName\nWHERE {\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n     VALUES (?chebi) { (CHEBI:32395) }\n     ?rhea rh:side/rh:contains/rh:compound ?compound .\n     ?compound rh:chebi ?chebi .\n     ?chebi up:name ?uniprotName .\n  }\n  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .\n  ?uniprot up:mnemonic ?uniprotID .\n  ?uniprot up:recommendedName/up:fullName ?recname .\n  OPTIONAL {?uniprot up:encodedBy/skos:prefLabel ?gene .}\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/51_all_proteins_linked_to_arachidonate.ttl",
    "filename": "51_all_proteins_linked_to_arachidonate.ttl"
  },
  
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/40",
    "natural_language_question": "Retrieve human enzymes that metabolize sphingolipids and are annotated in ChEMBL",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT DISTINCT ?protein ?chemblEntry\nWHERE {\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?rhea rdfs:subClassOf rh:Reaction ;\n      rh:side/rh:contains/rh:compound/rh:chebi/rdfs:subClassOf+ CHEBI:26739 .\n  }\n  ?ca up:catalyzedReaction ?rhea .\n  ?protein up:annotation/up:catalyticActivity ?ca ;\n    up:organism taxon:9606 ;\n    rdfs:seeAlso ?chemblEntry .\n  ?chemblEntry up:database <http://purl.uniprot.org/database/ChEMBL> .\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/40_human_enzymes_that_metabolize_sphingolipids.ttl",
    "filename": "40_human_enzymes_that_metabolize_sphingolipids.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/92_uniprot_bioregistry_iri_translation",
    "natural_language_question": "Translate the global unique identifier for a UniProtKB record into other options using the bioregistry translating endpoint.",
    "query": "PREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT \n    ?protein \n    ?otherIdentifier\nWHERE {\n    BIND(uniprotkb:P00750 AS ?protein)\n    ?protein a up:Protein .\n    SERVICE <https://bioregistry.io/sparql> {\n        ?protein owl:sameAs ?otherIdentifier .\n    }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://bioregistry.io/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/92_uniprot_bioregistry_iri_translation.ttl",
    "filename": "92_uniprot_bioregistry_iri_translation.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/49",
    "natural_language_question": "Retrieve the UniProtKB proteins, their catalyzed Rhea reactions, their encoding genes (Ensembl) and the anatomic entities where the genes are expressed (UBERON anatomic entites from Bgee expression data resource).",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX genex: <http://purl.org/genex#>\nPREFIX lscr: <http://purl.org/lscr#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT DISTINCT ?protein ?ensemblGene ?reaction ?anatomicEntityLabel ?anatomicEntity\nWHERE {\n  # federated query to Rhea enadpoint\n  {\n    SELECT DISTINCT ?reaction WHERE {\n      SERVICE <https://sparql.rhea-db.org/sparql> {\n        ?reaction rdfs:subClassOf rh:Reaction ;\n          rh:equation ?reactionEquation ;\n          rh:side ?reactionSide .\n        ?reactionSide rh:contains ?participant .\n        ?participant rh:compound ?compound .\n        # compound constraint (CHEBI:16113 == cholesterol)\n        ?compound rh:chebi CHEBI:16113 .\n      }\n    }\n  }\n  # taxonomy constraint (taxon:9606 == Homo sapiens)\n  ?protein up:organism taxon:9606 ;\n    up:annotation ?a ;\n    rdfs:seeAlso / up:transcribedFrom ?ensemblGene .\n  ?a a up:Catalytic_Activity_Annotation ;\n    up:catalyticActivity ?ca .\n  ?ca up:catalyzedReaction ?reaction .\n  # federated query to Bgee (expression data)\n  BIND(IRI(REPLACE(STR(?ensemblGene), \"\\\\.[0-9]+$\", \"\")) AS ?ensemblGeneNoVersion)\n  SERVICE <https://www.bgee.org/sparql/> {\n    ?gene lscr:xrefEnsemblGene ?ensemblGeneNoVersion ;\n      genex:isExpressedIn ?anatomicEntity .\n    ?anatomicEntity rdfs:label ?anatomicEntityLabel .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/49_tissues_where_genes_metabolizing_cholestrol_are_expressed.ttl",
    "filename": "49_tissues_where_genes_metabolizing_cholestrol_are_expressed.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/53",
    "natural_language_question": "Find mouse homologs in OMABrowser of human enzymes that catalyze reactions involving sterols (CHEBI:15889). Federating with Rhea and OMABrowser.",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX lscr: <http://purl.org/lscr#>\nPREFIX orth: <http://purl.org/net/orth#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  DISTINCT\n    ?chebi\n    ?reaction\n    ?humanProtein\n    ?mouseProtein\n    ?cluster \nWHERE {\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?reaction rdfs:subClassOf rh:Reaction .\n    ?reaction rh:side/rh:contains/rh:compound ?compound .\n    ?compound rh:chebi ?chebi .\n    ?chebi rdfs:subClassOf* CHEBI:15889\n  }\n\n  ?humanProtein up:organism taxon:9606 .\n  ?humanProtein up:annotation ?a .\n  ?a a up:Catalytic_Activity_Annotation .\n  ?a up:catalyticActivity ?ca .\n  ?ca up:catalyzedReaction ?reaction .\n\n  SERVICE  <https://sparql.omabrowser.org/sparql> {\n    ?cluster a orth:ParalogsCluster .\n    ?cluster orth:hasHomologousMember ?node1 , ?node2 .\n    ?node1 orth:hasHomologousMember* ?orthoProtein1 .\n    ?node2 orth:hasHomologousMember* ?orthoProtein2 .\n    ?orthoProtein1 lscr:xrefUniprot ?mouseProtein .\n    ?orthoProtein2 lscr:xrefUniprot ?humanProtein .\n    # inTaxon mouse\n    ?orthoProtein1 orth:organism/<http://purl.obolibrary.org/obo/RO_0002162> taxon:10090 . \n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/53_mouse_homologs_of_sterol_enzymes_via_omabrowser.ttl",
    "filename": "53_mouse_homologs_of_sterol_enzymes_via_omabrowser.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/52",
    "natural_language_question": "Retrieve drugs that target human enzymes involved in sterol metabolism (federated query with Rhea and ChEMBL via IDSM/Elixir czech republic).",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX cco: <http://rdf.ebi.ac.uk/terms/chembl#>\nPREFIX chebihash: <http://purl.obolibrary.org/obo/chebi#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n        DISTINCT\n            ?protein\n            ?proteinFullName\n            ?activityType\n            ?standardActivityValue\n            ?standardActivityUnit\n            ?chemblMolecule\n            ?chemlbMoleculePrefLabel\nWHERE\n  {\n  # ChEBI: retrieve members of the ChEBI class ChEBI:15889 (sterol)\n  # Rhea: retrieve the reactions involving these ChEBI as participants\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?reaction rdfs:subClassOf  rh:Reaction ;\n\t      rh:status        rh:Approved ;\n\t      rh:side          ?reactionSide .\n    ?reactionSide\n\t      rh:contains      ?participant .\n    ?participant rh:compound  ?compound\n    {\n      ?compound  rh:chebi  ?chebi .\n      ?chebi (rdfs:subClassOf)+ CHEBI:15889\n    } UNION {\n      ?compound  rh:chebi           ?chebi .\n      ?chebi2   rdfs:subClassOf     ?chebiRestriction .\n      ?chebiRestriction\n\t\ta           owl:Restriction ;\n\t\towl:onProperty      chebihash:has_major_microspecies_at_pH_7_3 ;\n\t\towl:someValuesFrom  ?chebi .\n      ?chebi2 (rdfs:subClassOf)+ CHEBI:15889\n    }\n  }\n  # UniProt: retrieve the human (taxid:9606) enzymes catalyzing these Rhea reactions\n  ?ca       up:catalyzedReaction  ?reaction .\n  ?a        up:catalyticActivity  ?ca .\n  ?protein  up:annotation         ?a ;\n\t    up:organism           taxon:9606 ;\n\t    up:recommendedName    ?proteinRecName .\n  ?proteinRecName\n\t    up:fullName           ?proteinFullName .\n  # Find drugs in wikidata that interact with the UniProt Proteins\n  # ChEMBL: retrieve the corresponding targets and with drugs in clinical phase 4\n  # Via https://idsm.elixir-czech.cz/sparql/\n  SERVICE <https://idsm.elixir-czech.cz/sparql/endpoint/idsm> {\n    ?activity a cco:Activity ;\n      cco:hasMolecule ?chemblMolecule ;\n      cco:hasAssay ?assay ;\n      cco:standardType ?activityType ;\n      cco:standardValue ?standardActivityValue ;\n      cco:standardUnits ?standardActivityUnit .\n    ?chemblMolecule cco:highestDevelopmentPhase ?highestDevelopmentPhase ;\n      rdfs:label ?chemblMoleculeLabel ;\n      skos:prefLabel ?chemlbMoleculePrefLabel .\n    FILTER (?highestDevelopmentPhase > 3)\n    ?assay cco:hasTarget ?target .\n    ?target cco:hasTargetComponent/cco:targetCmptXref ?protein .\n    ?protein a cco:UniprotRef .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/52_drugs_targeting_sterol_metabolism.ttl",
    "filename": "52_drugs_targeting_sterol_metabolism.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/109_uniprot_transporter_in_liver",
    "natural_language_question": "Find human transporter proteins in reviewed UniProtKB/Swiss-Prot, that are expressed in the liver (Uses Bgee and UBERON).",
    "query": "PREFIX genex: <http://purl.org/genex#>\nPREFIX lscr: <http://purl.org/lscr#>\nPREFIX obo: <http://purl.obolibrary.org/obo/>\nPREFIX orth: <http://purl.org/net/orth#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX uberon: <http://purl.obolibrary.org/obo/uo#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  ?rhea\n  ?protein\n  ?anat\nWHERE\n{\n  GRAPH <https://sparql.rhea-db.org/rhea> {\n    ?rhea rh:isTransport true .\n  }\n  ?protein up:annotation ?ann .\n  ?protein up:organism taxon:9606 .\n  ?ann up:catalyticActivity ?ca .\n  ?ca up:catalyzedReaction ?rhea .\n  BIND(uberon:0002107 AS ?anat)\n  SERVICE <https://www.bgee.org/sparql/> {\n    ?seq genex:isExpressedIn ?anat .\n    ?seq lscr:xrefUniprot ?protein .\n    ?seq orth:organism ?organism .\n    ?organism obo:RO_0002162 taxon:9606 .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://www.bgee.org/sparql/",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/109_uniprot_transporter_in_liver.ttl",
    "filename": "109_uniprot_transporter_in_liver.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/54",
    "natural_language_question": "Proteins with binding sites for ligands similar to heme",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX sachem: <http://bioinfo.uochb.cas.cz/rdf/v1.0/sachem#>\nPREFIX up: <http://purl.uniprot.org/core/>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT \n  ?protein\n  ?mnemonic\n  ?proteinName\n  ?ligandSimilarityScore\n  ?ligand\nWHERE {\n  SERVICE <https://idsm.elixir-czech.cz/sparql/endpoint/chebi> {\n    ?ssc sachem:compound ?ligand; \n      sachem:score ?ligandSimilarityScore ;\n      sachem:similaritySearch ?sss .\n        # Smiles of Heme\n    ?sss    sachem:query \"CC1=C(CCC([O-])=O)C2=[N+]3C1=Cc1c(C)c(C=C)c4C=C5C(C)=C(C=C)C6=[N+]5[Fe-]3(n14)n1c(=C6)c(C)c(CCC([O-])=O)c1=C2\";\n      sachem:cutoff \"8e-1\"^^xsd:double ;\n      sachem:aromaticityMode sachem:aromaticityDetect ;\n      sachem:similarityRadius 1 ;\n      sachem:tautomerMode sachem:ignoreTautomers .\n  }\n  ?protein up:mnemonic ?mnemonic ;\n    up:recommendedName/up:fullName ?proteinName ;\n    up:annotation ?annotation .\n  ?annotation a up:Binding_Site_Annotation ;\n      up:ligand/rdfs:subClassOf ?ligand .\n}\nORDER BY DESC(?ligandSimilarityScore)",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://idsm.elixir-czech.cz/sparql/endpoint/chebi",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/54_binding_sites_for_ligands_similar_to_heme_sachem.ttl",
    "filename": "54_binding_sites_for_ligands_similar_to_heme_sachem.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/43",
    "natural_language_question": "Connect patents cited in UniProtKB with those in the patent database at EPO via publication number, whose grant date is more than twenty years in the past.",
    "query": "PREFIX patent: <http://data.epo.org/linked-data/def/patent/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT ?grantDate ?patent ?application ?applicationNo\nWHERE\n{\n    ?citation a up:Patent_Citation ;\n  skos:exactMatch ?patent .\n  BIND(SUBSTR(STR(?patent), 35) AS ?applicationNo)\n  BIND(SUBSTR(STR(?patent), 33, 2) AS ?countryCode)\n  SERVICE <https://data.epo.org/linked-data/query>{\n    ?publication patent:publicationNumber ?applicationNo ;\n      patent:application ?application .\n    ?application patent:grantDate ?grantDate .\n  }\n  BIND((year(now()) - 20) AS ?thisYearMinusTwenty)\n  BIND(year(?grantDate) AS ?grantYear)\n  FILTER(?grantYear < ?thisYearMinusTwenty)\n} ORDER BY ?grantYear",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://data.epo.org/linked-data/query",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/43_patents_used_in_uniprot_granted_twenty_years_in_the_past.ttl",
    "filename": "43_patents_used_in_uniprot_granted_twenty_years_in_the_past.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/45",
    "natural_language_question": "Retrieve drugs that target human enzymes involved in sterol metabolism (federated query with WikiData and Rhea).",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX chebihash: <http://purl.obolibrary.org/obo/chebi#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>\nPREFIX up: <http://purl.uniprot.org/core/>\nPREFIX wdt: <http://www.wikidata.org/prop/direct/>\n\nSELECT DISTINCT ?protein ?proteinFullName ?wikiChemical ?wikiChemicalLabel ?medicalConditionTreatedLabel\nWHERE {\n  # ChEBI: retrieve members of the ChEBI class ChEBI:15889 (sterol)\n  # Rhea: retrieve the reactions involving these ChEBI as participants\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?reaction rdfs:subClassOf rh:Reaction ;\n      rh:status rh:Approved ;\n      rh:side ?reactionSide .\n    ?reactionSide\n      rh:contains ?participant .\n    ?participant rh:compound ?compound\n    {\n      ?compound rh:chebi ?chebi .\n      ?chebi (rdfs:subClassOf)+ CHEBI:15889\n    } UNION {\n      ?compound rh:chebi ?chebi .\n      ?chebi2 rdfs:subClassOf ?chebiRestriction .\n      ?chebiRestriction\n\t\ta owl:Restriction ;\n\t\towl:onProperty chebihash:has_major_microspecies_at_pH_7_3 ;\n\t\towl:someValuesFrom ?chebi .\n      ?chebi2 (rdfs:subClassOf)+ CHEBI:15889\n    }\n  }\n  # UniProt: retrieve the human (taxid:9606) enzymes catalyzing these Rhea reactions\n  ?ca up:catalyzedReaction  ?reaction .\n  ?a up:catalyticActivity  ?ca .\n  ?proteinIRI  up:annotation ?a ;\n    up:organism taxon:9606 ;\n    up:recommendedName ?proteinRecName .\n  ?proteinRecName up:fullName ?proteinFullName .\n  # Find drugs in wikidata that interact with the UniProt Proteins\n  BIND (SUBSTR(STR(?proteinIRI), STRLEN(STR(uniprotkb:))+1) AS ?protein)\n  SERVICE <https://query.wikidata.org/sparql> {\n    ?wp wdt:P352  ?protein .\n    ?wikiChemical wdt:P129 ?wp . # Physically interacts with\n    ?wikiChemical rdfs:label ?wikiChemicalLabel .\n    ?wikiChemical wdt:P2175 ?wmc . # Medical conndition treated\n    ?wmc rdfs:label ?medicalConditionTreatedLabel .\n    FILTER(lang(?medicalConditionTreatedLabel) = 'en')\n    FILTER(lang(?wikiChemicalLabel) = 'en')\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://query.wikidata.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/45_drugs_targeting_human_sterol_metabolism_enzymes.ttl",
    "filename": "45_drugs_targeting_human_sterol_metabolism_enzymes.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/42",
    "natural_language_question": "Connect patents cited in UniProtKB with those in the patent database at EPO via publication number.",
    "query": "PREFIX patent: <http://data.epo.org/linked-data/def/patent/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT ?citation ?patent ?application ?applicationNo\nWHERE\n{\n  ?citation a up:Patent_Citation ;\n    skos:exactMatch ?patent .\n  FILTER(CONTAINS(STR(?patent), 'EP'))\n  BIND(SUBSTR(STR(?patent), 35) AS ?applicationNo)\n  SERVICE <https://data.epo.org/linked-data/query>{\n    ?application patent:publicationNumber ?applicationNo\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://data.epo.org/linked-data/query",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/42_connect_patents_to_epo.ttl",
    "filename": "42_connect_patents_to_epo.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/67_draft_human_metabolome",
    "natural_language_question": "Generate a draft human metabolome",
    "query": "PREFIX chebislash: <http://purl.obolibrary.org/obo/chebi/>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n?uniprot ?mnemonic ?rhea ?chebi ?smiles ?inchiKey\nWHERE\n{\n  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea ;\n        up:organism taxon:9606 ;\n        up:mnemonic ?mnemonic .\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n\t?rhea rh:side/rh:contains/rh:compound ?compound .\n    ?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?chebi .\n   }\n   ?chebi chebislash:smiles ?smiles ;\n          chebislash:inchikey ?inchiKey .\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/67_draft_human_metabolome.ttl",
    "filename": "67_draft_human_metabolome.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/36",
    "natural_language_question": "Find the orthologous proteins for UniProtKB entry P05067 using the <a href=\"http://www.orthod.org\">OrthoDB database</a>",
    "query": "PREFIX orthodb: <http://purl.orthodb.org/>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  ?protein\n  ?orthoGroup\n  ?scientificName\n  ?functionComment\n  ?prefferedGeneName\n  ((STRLEN(?value) - ?medianLength) as ?deviationFromMedianLength)\nWHERE\n{\n  uniprotkb:P05067 a up:Protein ;\n        up:organism/up:scientificName ?scientificName ;\n        rdfs:seeAlso ?orthoGroup ;\n        up:encodedBy/skos:prefLabel ?prefferedGeneName ;\n          up:sequence/rdf:value ?value .\n  OPTIONAL {\n    ?protein up:annotation ?functionAnnotation .\n    ?functionAnnotation a up:Function_Annotation ;\n      rdfs:comment ?functionComment .\n  }\n  SERVICE <https://sparql.orthodb.org/sparql>{\n    ?orthoGroup orthodb:ogMedianProteinLength ?medianLength .\n    ?orthoGroup orthodb:hasMember ?xref .\n    ?xref orthodb:xref/orthodb:xrefResource uniprotkb:P05067 .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.orthodb.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/36_orthologous_proteins_via_orthodb.ttl",
    "filename": "36_orthologous_proteins_via_orthodb.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/60",
    "natural_language_question": "Find human proteins that catalyze reactions where substrates or product have a <a href=\"https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:35516\">Cholestane skeleton</a>",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX sachem: <http://bioinfo.uochb.cas.cz/rdf/v1.0/sachem#>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  DISTINCT\n    ?protein\n    ?disease\n    ?rhea\n    ?chebi\n    ?omim\nWHERE {\n    # Find complete ChEBIs with a Cholestane skeleton, via the Czech Elixir node IDSM Sachem chemical substructure search.\n    SERVICE <https://idsm.elixir-czech.cz/sparql/endpoint/chebi> {\n      ?chebi sachem:substructureSearch [\n        sachem:query\n\"[C@]12(CCC3CCCC[C@]3(C)[C@@]1([H])CC[C@]1(C)[C@@]([H])([C@@](C)([H])CCCC(C)C)CC[C@@]21[H])[H]\"\n].\n   }\n   # Use the fact that UniProt catalytic activities are annotated using Rhea \n   # Mapping the found ChEBIs to Rhea reactions\n   SERVICE <https://sparql.rhea-db.org/sparql>{\n     ?rhea rh:side/rh:contains/rh:compound/rdfs:subClassOf ?chebi .\n   }\n   # Match the found Rhea reactions with human UniProtKB proteins\n   ?protein up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .\n   ?protein up:organism taxon:9606 .\n   # Find only those human entries that have an annotated related disease, and optionaly map these to OMIM\n   ?protein up:annotation/up:disease ?disease .\n   OPTIONAL {\n     ?disease rdfs:seeAlso ?omim .\n     ?omim up:database <http://purl.uniprot.org/database/MIM>\n   }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://idsm.elixir-czech.cz/sparql/endpoint/chebi",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/60_enzymes_working_on_substrate_with_Cholestene_backbone.ttl",
    "filename": "60_enzymes_working_on_substrate_with_Cholestene_backbone.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/70_enzymes_interacting_with_molecules_similar_to_dopamine",
    "natural_language_question": "Look for enzymes catalyzing reactions with molecules similar to dopamine.",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX sachem: <http://bioinfo.uochb.cas.cz/rdf/v1.0/sachem#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  ?protein\n  ?rhea\n  ?chebi\nWHERE {\n  # Use the smiles of dopamine CHEBI:18243\n  SERVICE <https://idsm.elixir-czech.cz/sparql/endpoint/chebi>{\n    ?chebi sachem:similarCompoundSearch [\n      sachem:query \"NCCc1ccc(O)c(O)c1\" ] .\n  }\n  GRAPH<https://sparql.rhea-db.org/rhea>{\n    ?rhea rh:side/rh:contains/rh:compound ?compound .\n    ?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?chebi .\n  }\n  ?protein up:reviewed true ;\n    up:annotation ?caa .\n  ?caa up:catalyticActivity/up:catalyzedReaction ?rhea .\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://idsm.elixir-czech.cz/sparql/endpoint/chebi",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/70_enzymes_interacting_with_molecules_similar_to_dopamine.ttl",
    "filename": "70_enzymes_interacting_with_molecules_similar_to_dopamine.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/38",
    "natural_language_question": "For the human entry P05067 (Amyloid-beta precursor protein) find the gene start ends in WikiData",
    "query": "PREFIX p: <http://www.wikidata.org/prop/>\nPREFIX pq: <http://www.wikidata.org/prop/qualifier/>\nPREFIX ps: <http://www.wikidata.org/prop/statement/>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>\nPREFIX wdt: <http://www.wikidata.org/prop/direct/>\n\n\nSELECT \n\t?protein \n\t?begin\n\t?end\n\t?chromosome\n\t?assembly\nWHERE {\n    {\n        BIND(uniprotkb:P05067 AS ?proteinIRI)\n        BIND (SUBSTR(STR(?proteinIRI), STRLEN(STR(uniprotkb:))+1) AS ?protein)\n    }\n    SERVICE <https://query.wikidata.org/sparql> {\n        ?wp wdt:P352 ?protein ;\n            wdt:P702 ?wg . \n        ?wg p:P644   ?wgss .\n        ?wgss ps:P644        ?begin ;\n          pq:P1057/wdt:P1813 ?chromosome ;\n          pq:P659/rdfs:label ?assembly .\n        ?wg p:P645 ?wgse .\n        ?wgse ps:P645        ?end ;\n          pq:P1057/wdt:P1813 ?chromosome ;\n          pq:P659/rdfs:label ?assembly .\n        FILTER(lang(?assembly) = \"en\")\n  } \n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://query.wikidata.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/38_find_gene_region_in_wikidata_for_known_entry.ttl",
    "filename": "38_find_gene_region_in_wikidata_for_known_entry.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/117_biosodafrontend_glioblastoma_orthologs_rat",
    "natural_language_question": "Which are the proteins associated with glioblastoma and the orthologs expressed in the rat brain?",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX obo: <http://purl.obolibrary.org/obo/>\nPREFIX orth: <http://purl.org/net/orth#>\nPREFIX sio: <http://semanticscience.org/resource/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\nPREFIX lscr: <http://purl.org/lscr#>\nPREFIX genex: <http://purl.org/genex#>\nSELECT DISTINCT ?protein ?orthologous_protein ?gene ?annotation_text WHERE {\n  {\n  \tSELECT ?protein ?annotation_text WHERE {\n      ?protein a up:Protein ;\n          up:organism taxon:9606 ;\n          up:annotation ?annotation .\n      ?annotation rdfs:comment ?annotation_text .\n      ?annotation a up:Disease_Annotation .\n      FILTER CONTAINS (?annotation_text, \"glioblastoma\")\n    }\n  }\n  SERVICE <https://sparql.omabrowser.org/sparql/> {\n    SELECT ?orthologous_protein ?protein ?gene WHERE {\n    ?protein_OMA a orth:Protein .\n    ?orthologous_protein a orth:Protein .\n    ?cluster a orth:OrthologsCluster .\n    ?cluster orth:hasHomologousMember ?node1 .\n    ?cluster\n    orth:hasHomologousMember ?node2 .\n    ?node2 orth:hasHomologousMember* ?protein_OMA .\n    ?node1 orth:hasHomologousMember* ?orthologous_protein .\n    ?orthologous_protein orth:organism/obo:RO_0002162 taxon:10116 . # rattus norvegicus\n    ?orthologous_protein sio:SIO_010079 ?gene .\n    ?protein_OMA lscr:xrefUniprot ?protein .\n    FILTER(?node1 != ?node2)\n\t\t}\n\t}\n  SERVICE <https://www.bgee.org/sparql/> {\n    ?gene genex:isExpressedIn ?a .\n    ?a rdfs:label \"brain\" .\n    ?gene orth:organism ?s . \n    ?s obo:RO_0002162 taxon:10116.\n\t}\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql",
    "federates_with": "https://sparql.omabrowser.org/sparql/",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/117_biosodafrontend_glioblastoma_orthologs_rat.ttl",
    "filename": "117_biosodafrontend_glioblastoma_orthologs_rat.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/116_biosodafrontend_rabit_mouse_orthologs",
    "natural_language_question": "Rabbit's proteins encoded by genes that are orthologous to Mouse's HBB-Y gene and their cross reference links to Uniprot",
    "query": "PREFIX lscr: <http://purl.org/lscr#>\nPREFIX orth: <http://purl.org/net/orth#>\nPREFIX up: <http://purl.uniprot.org/core/>\nPREFIX obo: <http://purl.obolibrary.org/obo/>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nSELECT DISTINCT ?PROTEIN_1 ?PROTEIN_2 ?UNIPROT_XREF_1 ?UNIPROT_XREF_2 WHERE {\n\t?taxon_1 up:commonName 'Mouse' .\n\t?taxon_2 up:commonName 'Rabbit' .\n\tSERVICE <https://sparql.omabrowser.org/sparql/> {\n\t\t?cluster a orth:OrthologsCluster .\n\t\t?cluster orth:hasHomologousMember ?node1 .\n\t\t?cluster orth:hasHomologousMember ?node2 .\n\t\t?node2 orth:hasHomologousMember* ?PROTEIN_2 .\n\t\t?node1 orth:hasHomologousMember* ?PROTEIN_1 .\n\t\t?PROTEIN_1 a orth:Protein .\n\t\t?PROTEIN_1 orth:organism/obo:RO_0002162 ?taxon_1 ;\n\t\t\trdfs:label 'HBB-Y' ;\n\t\t\tlscr:xrefUniprot ?UNIPROT_XREF_1 .\n\t\t?PROTEIN_2 a orth:Protein .\n\t\t?PROTEIN_2 orth:organism/obo:RO_0002162 ?taxon_2 .\n\t\t?PROTEIN_2 lscr:xrefUniprot ?UNIPROT_XREF_2 .\n\t\tFILTER ( ?node1 != ?node2 )\n\t}\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql",
    "federates_with": "https://sparql.omabrowser.org/sparql/",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/116_biosodafrontend_rabit_mouse_orthologs.ttl",
    "filename": "116_biosodafrontend_rabit_mouse_orthologs.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/29",
    "natural_language_question": "Use <a href=\"http://allie.dbcls.jp/\">ALLIE</a> a service for Abbreviation / Long Form in Japanese and English to search in UniProt using Japanese.",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT ?protein ?englishLabelStr\nWHERE {\n    SERVICE <https://data.allie.dbcls.jp/sparql>{\n        ?x rdfs:label \"アミロイド前駆体タンパク質\"@ja ;\n            rdfs:label ?englishLabel .\n        FILTER(lang(?englishLabel) = \"en\")\n    }\n    BIND (STR(?englishLabel) AS ?englishLabelStr)\n    ?protein a up:Protein .\n    {\n        ?protein (up:recommendedName|up:alternativeName) ?structuredName .\n    }\n    UNION\n    {\n        VALUES(?partType){(up:domain) (up:component)}\n            ?protein ?partType ?part .\n        ?part (up:recommendedName|up:alternativeName) ?structuredName .\n    }\n    ?structuredName ?anyKindOfName  ?englishLabelStr .\n    ?anyKindOfName rdfs:subPropertyOf up:structuredNameType .\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://data.allie.dbcls.jp/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/29_search_uniprot_in_japanese_with_allie.ttl",
    "filename": "29_search_uniprot_in_japanese_with_allie.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/94_Select_all_approved_reactions_with_CHEBI_or_one_of_its_descendant_optional_UniProtKB_reviewed_EC",
    "natural_language_question": "Select all approved reactions with CHEBI:17815 (a 1,2-diacyl-sn-glycerol) or one of its descendant. Display links to UniProtKB/Swiss-Prot entries via EC numbers if the link rhea-ec exists.",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh:<http://rdf.rhea-db.org/>\nPREFIX CHEBI:<http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX up:<http://purl.uniprot.org/core/>\n\nSELECT\n  distinct\n    ?reaction\n    ?enzyme\n    (count(distinct ?protein) as ?proteinCount)\nWHERE {\n  ?reaction rdfs:subClassOf rh:Reaction .\n  ?reaction rh:status rh:Approved .\n  ?reaction rh:side ?reactionSide .\n\n  OPTIONAL {?reaction rh:ec ?enzyme .\n    SERVICE <https://sparql.uniprot.org/sparql> {\n      ?protein up:reviewed true .\n      ?protein up:enzyme ?enzyme.\n    }\n  } .\n  ?reactionSide rh:contains ?participant .\n  ?participant rh:compound ?compound .\n  ?compound rh:chebi ?chebi .\n  ?chebi rdfs:subClassOf* CHEBI:17815 .\n}\nGROUP BY ?reaction ?enzyme\nORDER BY ?reaction",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/94_Select_all_approved_reactions_with_CHEBI_or_one_of_its_descendant_optional_UniProtKB_reviewed_EC.ttl",
    "filename": "94_Select_all_approved_reactions_with_CHEBI_or_one_of_its_descendant_optional_UniProtKB_reviewed_EC.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/11",
    "natural_language_question": "Distribution of reactions according to the first class of the enzyme classification (federated query)",
    "query": "PREFIX ec: <http://purl.uniprot.org/enzyme/>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\n\n# Query 11\n# Retrieve the count of reactions mapped to each level (main class) of the enzyme classification\n#\n# This query mimics the Filter section of the Rhea website (Browse all reactions)\n# https://www.rhea-db.org/rhea?query=\nSELECT ?ecClass (STR(?ecName) AS ?ecClassName) (COUNT(?rhea) AS ?rheaCount)\nWHERE {\n  SERVICE <https://sparql.uniprot.org/sparql> {\n    VALUES (?ecClass) { (ec:1.-.-.-)(ec:2.-.-.-)(ec:3.-.-.-)(ec:4.-.-.-)(ec:5.-.-.-) (ec:6.-.-.-) (ec:7.-.-.-)}\n    ?ecNumber rdfs:subClassOf ?ecClass .\n    ?ecClass skos:prefLabel ?ecName .\n  }\n  ?rhea rdfs:subClassOf rh:Reaction .\n  ?rhea rh:ec ?ecNumber .\n} GROUP BY ?ecClass ?ecName",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/11.ttl",
    "filename": "11.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/109_Distribution_of_reactions_according_to_the_first_level_of_enzyme_classification",
    "natural_language_question": "Distribution of reactions according to the first level of enzyme classification",
    "query": "\nPREFIX rh:<http://rdf.rhea-db.org/>\nPREFIX ec:<http://purl.uniprot.org/enzyme/>\nPREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>\nPREFIX skos:<http://www.w3.org/2004/02/skos/core#>\n\nSELECT\n  ?ecClass\n  (str(?ecName) as ?ecClassName)\n  (count(?reaction) as ?reactionCount)\nWHERE\n{\n  SERVICE <http://sparql.uniprot.org/sparql> {\n    VALUES (?ecClass) { (ec:1.-.-.-)(ec:2.-.-.-)(ec:3.-.-.-)(ec:4.-.-.-)(ec:5.-.-.-) (ec:6.-.-.-) (ec:7.-.-.-) }\n    ?ecNumber rdfs:subClassOf ?ecClass .\n    ?ecClass skos:prefLabel ?ecName .\n  }\n  ?reaction rdfs:subClassOf rh:Reaction .\n  ?reaction rh:ec ?ecNumber .\n}\nGROUP BY ?ecClass ?ecName\n",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "http://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/109_Distribution_of_reactions_according_to_the_first_level_of_enzyme_classification.ttl",
    "filename": "109_Distribution_of_reactions_according_to_the_first_level_of_enzyme_classification.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/12",
    "natural_language_question": "Select all Rhea reactions linked to protein sequences (enzymes and transporters) in UniProtKB",
    "query": "PREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\n# Query 12\n# Select all Rhea reactions used to annotate enzyme sequences in UniProtKB\n# return the number of UniProtKB entries\n# Federated query using a service to UniProt SPARQL endpoint\n#\n# This query corresponds to the Rhea website query:\n# https://www.rhea-db.org/rhea?query=uniprot:*\n#\nSELECT ?uniprotCount ?rhea ?accession ?equation \nWHERE {\n  SERVICE <https://sparql.uniprot.org/sparql> { \n  \tSELECT ?rhea (count(?uniprot) as ?uniprotCount) {\n      ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea . \n  \t}\n  \tGROUP BY ?rhea\n  }\n  ?rhea rh:accession ?accession .\n  ?rhea rh:equation ?equation .\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/12.ttl",
    "filename": "12.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/112_Total_number_of_reaction_ec_protein_links",
    "natural_language_question": "Total number of <reaction> <ec> <protein> links",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh:<http://rdf.rhea-db.org/>\nPREFIX ec:<http://purl.uniprot.org/enzyme/>\nPREFIX up:<http://purl.uniprot.org/core/>\n\nSELECT\n  (count(?reaction) as ?reactionEcProteinLinkCount)\nWHERE {\n  ?reaction rdfs:subClassOf rh:Reaction .\n  ?reaction rh:ec ?enzyme\n  SERVICE <http://sparql.uniprot.org/sparql> {\n    ?protein up:reviewed true .\n    ?protein up:enzyme ?enzyme.\n  }\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "http://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/112_Total_number_of_reaction_ec_protein_links.ttl",
    "filename": "112_Total_number_of_reaction_ec_protein_links.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/19_draft_human_metabolome",
    "natural_language_question": "Generate a draft human metabolome",
    "query": "PREFIX chebislash: <http://purl.obolibrary.org/obo/chebi/>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  ?uniprot ?mnemonic ?rhea ?chebi ?smiles ?inchiKey\nWHERE\n{\n  ?rhea rh:side/rh:contains/rh:compound ?compound .\n  ?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?chebi .\n  ?chebi chebislash:smiles ?smiles ;\n          chebislash:inchikey ?inchiKey .\n  SERVICE <https://sparql.uniprot.org/sparql> {\n     ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea ;\n                                             up:organism taxon:9606 ;\n                                             up:mnemonic ?mnemonic .\n\n   }\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql/",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/19_draft_human_metabolome.ttl",
    "filename": "19_draft_human_metabolome.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/20_search_chemical_names_in_japanese",
    "natural_language_question": "Search for Chemical names starting in Japanese, translated to english using allie",
    "query": "PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n\nSELECT\n*\nWHERE {\nSERVICE <https://data.allie.dbcls.jp/sparql>{\n        ?x rdfs:label \"1,2,4-トリクロロベンゼン\"@ja ;\n            rdfs:label ?englishLabel .\n        FILTER(lang(?englishLabel) = \"en\")\n    }\n  BIND(STR(?englishLabel) AS ?englishLabelStr)\n  ?chebi rdfs:label|oboInOwl:hasSynonym ?englishLabelStr .\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql/",
    "federates_with": "https://data.allie.dbcls.jp/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/20_search_chemical_names_in_japanese.ttl",
    "filename": "20_search_chemical_names_in_japanese.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/117_Retrieve_parent_reactions_and_their_catalyzing_enzymes_for_a_metabolite_that_is_not_directly_involved_in_a_reaction",
    "natural_language_question": "Retrieve parent reactions (and their catalyzing enzymes) for a metabolite that is not directly involved in a reaction (ex: CHEBI:83137).",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh:<http://rdf.rhea-db.org/>\nPREFIX ec:<http://purl.uniprot.org/enzyme/>\nPREFIX up:<http://purl.uniprot.org/core/>\nPREFIX taxon:<http://purl.uniprot.org/taxonomy/>\nPREFIX keywords:<http://purl.uniprot.org/keywords/>\n\nSELECT\n  ?chebi\n  ?ecNumber\n  (count(?protein) as ?proteinCount)\n  ?reaction\n  ?equation\nWHERE {\n  ?reaction rdfs:subClassOf rh:Reaction .\n  ?reaction rh:status rh:Approved .\n  ?reaction rh:equation ?equation .\n  ?reaction rh:ec ?ecNumber .\n  ?reaction rh:side ?reactionSide .\n  ?reactionSide rh:contains ?participant .\n  ?participant rh:compound ?compound .\n  ?compound rh:chebi ?chebi .\n  CHEBI:83137 rdfs:subClassOf+ ?chebi.\n\n  SERVICE <https://sparql.uniprot.org/sparql> {\n  ?protein a up:Protein .\n  ?protein up:reviewed true .\n    {?protein up:enzyme ?ecNumber}\n  UNION\n    {?protein up:domain/up:enzyme ?ecNumber}\n  UNION\n    {?protein up:component/up:enzyme ?ecNumber} .\n  }\n} \nGROUP BY ?chebi ?ecNumber ?reaction ?equation",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/117_Retrieve_parent_reactions_and_their_catalyzing_enzymes_for_a_metabolite_that_is_not_directly_involved_in_a_reaction.ttl",
    "filename": "117_Retrieve_parent_reactions_and_their_catalyzing_enzymes_for_a_metabolite_that_is_not_directly_involved_in_a_reaction.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/18",
    "natural_language_question": "Use <a href=\"https://idsm.elixir-czech.cz/sparql\">IDSM Sachem</a> to find ChEBIs with a a <a href=\"https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:35516\">Cholestane</a> skeleton (in SMILES). Then match returned ChEBIs to Rhea undirected reactions.",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX sachem: <http://bioinfo.uochb.cas.cz/rdf/v1.0/sachem#>\n\n\nSELECT \n  ?rhea \n  ?chebi\nWHERE {\n  SERVICE <https://idsm.elixir-czech.cz/sparql/endpoint/chebi> {\n    ?chebi sachem:substructureSearch [\n        sachem:query \"[C@]12(CCC3CCCC[C@]3(C)[C@@]1([H])CC[C@]1(C)[C@@]([H])([C@@](C)([H])CCCC(C)C)CC[C@@]21[H])[H]\" ].\n  }\n  ?rhea rh:side/rh:contains/rh:compound/rdfs:subClassOf ?chebi .\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://idsm.elixir-czech.cz/sparql/endpoint/chebi",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/18.ttl",
    "filename": "18.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/115_Retrieve_the_set_of_metabolites_involved_in_reactions_catalyzed_by_human_genes_HUMAN_reference_proteome",
    "natural_language_question": "Retrieve the set of metabolites involved in reactions catalyzed by human genes (HUMAN reference proteome)",
    "query": "\nPREFIX rh:<http://rdf.rhea-db.org/>\nPREFIX taxon:<http://purl.uniprot.org/taxonomy/>\nPREFIX ec:<http://purl.uniprot.org/enzyme/>\nPREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>\nPREFIX up:<http://purl.uniprot.org/core/>\nPREFIX keywords:<http://purl.uniprot.org/keywords/>\n\nSELECT\n  ?chebi\n  ?reaction\n  ?ecNumber\n  ?protein\n  ?ensemblTranscript\n  ?ensemblGene\nWHERE {\n  SERVICE <https://sparql.uniprot.org/sparql> {\n    ?protein a up:Protein .\n    ?protein up:reviewed true .\n    ?protein up:organism taxon:9606 .\n    ?protein up:classifiedWith keywords:1185 .\n\n    ?protein rdfs:seeAlso ?ensemblTranscript .\n    ?ensemblTranscript a up:Transcript_Resource .\n    ?ensemblTranscript up:database <http://purl.uniprot.org/database/Ensembl> .\n    ?ensemblTranscript up:transcribedFrom ?ensemblGene .\n\n      {?protein up:enzyme ?ecNumber .}\n    UNION\n      {?protein up:domain/up:enzyme ?ecNumber .}\n    UNION\n      {?protein up:component/up:enzyme ?ecNumber .}\n  }\n\n  ?reaction rdfs:subClassOf rh:Reaction .\n  ?reaction rh:status rh:Approved .\n  ?reaction rh:ec ?ecNumber .\n  ?reaction rh:side ?reactionSide .\n  ?reactionSide rh:contains ?participant .\n  ?participant rh:compound ?compound .\n  ?compound rh:chebi ?chebi . # only considering small molecules participants\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/115_Retrieve_the_set_of_metabolites_involved_in_reactions_catalyzed_by_human_genes_HUMAN_reference_proteome.ttl",
    "filename": "115_Retrieve_the_set_of_metabolites_involved_in_reactions_catalyzed_by_human_genes_HUMAN_reference_proteome.ttl"
  },
  {
    "resource": "https://sparql.swisslipids.org/.well-known/sparql-examples/16",
    "natural_language_question": "For a given lipid (SWISSLIPID:000399814, Ceramide), return a list of all enzymes in UniProtKB/Swiss-Prot annotated to catalyze reactions involving that lipid.\nThis query does not consider the hierarchy: it looks for reactions and enzymes only for the specified lipid.",
    "query": "PREFIX SWISSLIPID: <https://swisslipids.org/rdf/SLM_>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\n# Example 16\nSELECT ?startId ?startName ?uniprot\nWHERE\n{\n  # SwissLipids query (list of) identifier(s)\n  VALUES ?startId { SWISSLIPID:000399814 }\n  # name\n  ?startId rdfs:label ?startName .\n  # ChEBI x-ref\n  ?startId owl:equivalentClass ?chebi .\n  # federated query to Rhea\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n  \t?rhea rh:side/rh:contains/rh:compound/rh:chebi ?chebi .\n  }    \n  # federated query to UniProt\n  SERVICE <https://sparql.uniprot.org/sparql> {\n    ?uniprot up:reviewed true .\n    ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .\n  }\n} \nORDER BY ?startId",
    "target_endpoint": "https://sparql.swisslipids.org/sparql/",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "SwissLipids",
    "file_path": "SwissLipids/16.ttl",
    "filename": "16.ttl"
  },
  {
    "resource": "https://sparql.swisslipids.org/.well-known/sparql-examples/6",
    "natural_language_question": "For a given lipid (SWISSLIPID:000399814, Ceramide), return a list of all corresponding reactions (Rhea IDs and equation), as well as their enzymes as annotated in UniProtKB/Swiss-Prot) enzymes themselves (ID and protein name).\nThis query does not consider the hierarchy: it looks for reactions and enzymes only for the specified lipid.",
    "query": "PREFIX SWISSLIPID: <https://swisslipids.org/rdf/SLM_>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\n# Example 6\nSELECT ?startId ?startName ?chebi ?rhea ?rheaEquation ?uniprot ?uniprotName\nWHERE\n{\n  # SwissLipids query (list of) identifier(s)\n  VALUES ?startId { SWISSLIPID:000399814 }\n  # name\n  ?startId rdfs:label ?startName .\n  # ChEBI x-ref\n  ?startId owl:equivalentClass ?chebi .\n  # federated query to Rhea\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n  \t?rhea rh:equation ?rheaEquation .\n  \t?rhea rh:side/rh:contains/rh:compound/rh:chebi ?chebi .\n  }    \n  # federated query to UniProt\n  SERVICE <https://sparql.uniprot.org/sparql> {\n    ?uniprot up:reviewed true .\n    ?uniprot up:recommendedName/up:fullName ?uniprotName .\n    ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .\n  }\n} \nORDER BY ?startId",
    "target_endpoint": "https://sparql.swisslipids.org/sparql/",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "SwissLipids",
    "file_path": "SwissLipids/6.ttl",
    "filename": "6.ttl"
  },
  {
    "resource": "https://sparql.swisslipids.org/.well-known/sparql-examples/5",
    "natural_language_question": "For a given lipid (SWISSLIPID:000399814, Ceramide), return a list of all corresponding reactions (Rhea IDs and equation).\nThis query does consider the hierarchy: it looks for reactions involving the specified lipid and/or its more specific forms.",
    "query": "PREFIX SWISSLIPID: <https://swisslipids.org/rdf/SLM_>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\n\n# Example 5\nSELECT ?startId ?startName ?id ?name ?chebi ?rhea ?rheaEquation \nWHERE\n{\n  # SwissLipids query (list of) identifier(s)\n  VALUES ?startId { SWISSLIPID:000399814 }\n  # name\n  ?startId rdfs:label ?startName .\n  # Retrieve children lipids \n  ?id rdfs:subClassOf* ?startId .\n  ?id rdfs:label ?name .\n  # ChEBI x-ref\n  ?id owl:equivalentClass ?chebi .\n  # federated query to Rhea\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n  \t?rhea rh:equation ?rheaEquation .\n  \t?rhea rh:side/rh:contains/rh:compound/rh:chebi ?chebi .\n  }    \n} \nORDER BY  ?startId ?id",
    "target_endpoint": "https://sparql.swisslipids.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "SwissLipids",
    "file_path": "SwissLipids/5.ttl",
    "filename": "5.ttl"
  },
  {
    "resource": "https://sparql.swisslipids.org/.well-known/sparql-examples/34",
    "natural_language_question": "Lipids affected by human enzymes",
    "query": "PREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT ?swisslipid  {\n  ?swisslipid owl:equivalentClass ?chebi .\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?rhea rh:side/rh:contains/rh:compound ?compound .\n\t?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?metabolite . \n  }\n  SERVICE <https://sparql.uniprot.org/sparql> {\n    ?catalyticActivityAnnotation up:catalyticActivity/up:catalyzedReaction ?rhea .\n    ?protein up:annotation ?catalyticActivityAnnotation ;\n             up:organism taxon:9606 .\n  }\n}",
    "target_endpoint": "https://sparql.swisslipids.org/sparql/",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "SwissLipids",
    "file_path": "SwissLipids/34.ttl",
    "filename": "34.ttl"
  },
]

In [4]:

import sys
import os

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from entity_indexing.endpoint_loader import query_sparql

error_list = []
no_result_list = []
count = 0

for object in test_set:
    print(f"Querying endpoint {object['filename']}...")
    print(f"arrow {count}/{len(test_set)}")
    result = query_sparql(object['query'] + " LIMIT 1", object['target_endpoint'])
    count += 1
    if result == 'error':
        print(f"Error querying endpoint {object['filename']}")
        error_list.append({"filename": object['filename'],"query": object['query'],"endpoint": object['target_endpoint'], "result": "error"})
    else:
        if not result.get("results") or not result["results"].get("bindings") or len(result["results"]["bindings"]) == 0:
            print(f"No results for {object['filename']}")
            no_result_list.append({
                "filename": object['filename'],
                "query": object['query'],
                "endpoint": object['target_endpoint'],
                "result": "no_results"
            })


Querying endpoint 118_biosodafrontend_rat_brain_human_cancer.ttl...
arrow 0/35
Querying endpoint 50_tissues_where_genes_sphingosines_are_expressed.ttl...
arrow 1/35
No results for 50_tissues_where_genes_sphingosines_are_expressed.ttl
Querying endpoint 51_all_proteins_linked_to_arachidonate.ttl...
arrow 2/35
Querying endpoint 40_human_enzymes_that_metabolize_sphingolipids.ttl...
arrow 3/35
Querying endpoint 92_uniprot_bioregistry_iri_translation.ttl...
arrow 4/35
No results for 92_uniprot_bioregistry_iri_translation.ttl
Querying endpoint 49_tissues_where_genes_metabolizing_cholestrol_are_expressed.ttl...
arrow 5/35
Querying endpoint 53_mouse_homologs_of_sterol_enzymes_via_omabrowser.ttl...
arrow 6/35
Querying endpoint 52_drugs_targeting_sterol_metabolism.ttl...
arrow 7/35
Querying endpoint 109_uniprot_transporter_in_liver.ttl...
arrow 8/35
No results for 109_uniprot_transporter_in_liver.ttl
Querying endpoint 54_binding_sites_for_ligands_similar_to_heme_sachem.ttl...
arrow 9/35
Querying 

In [5]:
print(len(error_list))

1


In [7]:
import pandas as pd
df = pd.DataFrame(error_list)
df

,filename,query,endpoint,result
0,43_patents_used_in_uniprot_granted_twenty_year...,PREFIX patent: <http://data.epo.org/linked-dat...,https://sparql.uniprot.org/sparql/,error


In [6]:
import pandas as pd
df = pd.DataFrame(no_result_list)
df


,filename,query,endpoint,result
0,50_tissues_where_genes_sphingosines_are_expres...,PREFIX CHEBI: <http://purl.obolibrary.org/obo/...,https://sparql.uniprot.org/sparql/,no_results
1,92_uniprot_bioregistry_iri_translation.ttl,PREFIX owl: <http://www.w3.org/2002/07/owl#>\n...,https://sparql.uniprot.org/sparql/,no_results
2,109_uniprot_transporter_in_liver.ttl,PREFIX genex: <http://purl.org/genex#>\nPREFIX...,https://sparql.uniprot.org/sparql/,no_results


In [48]:
df_ground_truth_errors= pd.DataFrame(error_list)

In [49]:
df_ground_truth_errors

,filename,query,endpoint,result
0,46_duck_pictures_via_eepa_and_taxonomy.ttl,PREFIX eunisSpecies: <http://eunis.eea.europa....,https://sparql.uniprot.org/sparql/,error
1,90_uniprot_affected_by_metabolic_diseases_usin...,PREFIX rdfs: <http://www.w3.org/2000/01/rdf-sc...,https://sparql.uniprot.org/sparql/,error
2,48_glycosylation_sites_and_glycans.ttl,PREFIX faldo: <http://biohackathon.org/resourc...,https://sparql.uniprot.org/sparql/,error
3,99_uniprot_identifiers_org_translation.ttl,PREFIX owl: <http://www.w3.org/2002/07/owl#>\n...,https://sparql.uniprot.org/sparql/,error
4,71_enzymes_interacting_with_molecules_similar_...,PREFIX CHEBI: <http://purl.obolibrary.org/obo/...,https://sparql.uniprot.org/sparql/,error
5,13.ttl,PREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX t...,https://sparql.rhea-db.org/sparql,error
6,7.ttl,PREFIX SWISSLIPID: <https://swisslipids.org/rd...,https://sparql.swisslipids.org/sparql/,error
7,27.ttl,PREFIX SWISSLIPID: <https://swisslipids.org/rd...,https://sparql.swisslipids.org/sparql/,error
8,26.ttl,PREFIX owl: <http://www.w3.org/2002/07/owl#>\n...,https://sparql.swisslipids.org/sparql/,error


In [2]:
import sys
import os


sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.query_cache import cache_dataset_queries


result = cache_dataset_queries(
    endpoint_files_map={
        "UniProt": ["67_draft_human_metabolome.ttl"]
    }
)

print(result)

Processing endpoint set: UniProt
  Processing 1 specified files
  Processing TTL file: 67_draft_human_metabolome.ttl

  Using cached result for query

Caching complete. Total queries cached: 1
  UniProt: 1 queries
{'UniProt': 1}


In [1]:
import sys
import os
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.query_cache import query_sparql

test = {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/67_draft_human_metabolome",
    "natural_language_question": "Generate a draft human metabolome",
    "query": "PREFIX chebislash: <http://purl.obolibrary.org/obo/chebi/>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n?uniprot ?mnemonic ?rhea ?chebi ?smiles ?inchiKey\nWHERE\n{\n  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea ;\n        up:organism taxon:9606 ;\n        up:mnemonic ?mnemonic .\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n\t?rhea rh:side/rh:contains/rh:compound ?compound .\n    ?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?chebi .\n   }\n   ?chebi chebislash:smiles ?smiles ;\n          chebislash:inchikey ?inchiKey .\n}\nLIMIT 10",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/67_draft_human_metabolome.ttl",
    "filename": "67_draft_human_metabolome.ttl"
  }

result = query_sparql(test['query'], test['target_endpoint'])

print(result)


{'head': {'vars': ['uniprot', 'mnemonic', 'rhea', 'chebi', 'smiles', 'inchiKey']}, 'results': {'bindings': [{'inchiKey': {'type': 'literal', 'value': 'XPPKVPWEQAFLFU-UHFFFAOYSA-K'}, 'chebi': {'type': 'uri', 'value': 'http://purl.obolibrary.org/obo/CHEBI_33019'}, 'smiles': {'type': 'literal', 'value': 'OP([O-])(=O)OP([O-])([O-])=O'}, 'rhea': {'type': 'uri', 'value': 'http://rdf.rhea-db.org/22508'}, 'mnemonic': {'type': 'literal', 'value': 'A0A087WW42_HUMAN'}, 'uniprot': {'type': 'uri', 'value': 'http://purl.uniprot.org/uniprot/A0A087WW42'}}, {'inchiKey': {'type': 'literal', 'value': 'XLYOFNOQVPJJNP-UHFFFAOYSA-N'}, 'chebi': {'type': 'uri', 'value': 'http://purl.obolibrary.org/obo/CHEBI_15377'}, 'smiles': {'type': 'literal', 'value': '[H]O[H]'}, 'rhea': {'type': 'uri', 'value': 'http://rdf.rhea-db.org/14777'}, 'mnemonic': {'type': 'literal', 'value': 'AMPD2_HUMAN'}, 'uniprot': {'type': 'uri', 'value': 'http://purl.uniprot.org/uniprot/Q01433'}}, {'inchiKey': {'type': 'literal', 'value': 'Q

In [30]:
ground_truth_query = """PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX rh: <http://rdf.rhea-db.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX up: <http://purl.uniprot.org/core/>


SELECT 
    DISTINCT
        ?uniprot
        ?uniprotID
        ?recname
        ?gene
        ?chebi
        ?uniprotName
WHERE {
  SERVICE <https://sparql.rhea-db.org/sparql> {
     VALUES (?chebi) { (CHEBI:32395) }
     ?rhea rh:side/rh:contains/rh:compound ?compound .
     ?compound rh:chebi ?chebi .
     ?chebi up:name ?uniprotName .
  }
  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .
  ?uniprot up:mnemonic ?uniprotID .
  ?uniprot up:recommendedName/up:fullName ?recname .
  OPTIONAL {?uniprot up:encodedBy/skos:prefLabel ?gene .}
}
LIMIT 100
"""

ground_truth_endpoint = "https://sparql.uniprot.org/sparql/"

predicted_query = """PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX rh: <http://rdf.rhea-db.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX up: <http://purl.uniprot.org/core/>


SELECT 
    DISTINCT
        ?uniprot
        ?uniprotID
        ?recname
WHERE {
  SERVICE <https://sparql.rhea-db.org/sparql> {
     VALUES (?chebi) { (CHEBI:32395) }
     ?rhea rh:side/rh:contains/rh:compound ?compound .
     ?compound rh:chebi ?chebi .
     ?chebi up:name ?uniprotName .
  }
  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .
  ?uniprot up:mnemonic ?uniprotID .
  ?uniprot up:recommendedName/up:fullName ?recname .
  OPTIONAL {?uniprot up:encodedBy/skos:prefLabel ?gene .}
}
LIMIT 100
"""
predicted_endpoint = ground_truth_endpoint

In [31]:
import sys
import os
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')
from experiments.utilities.result_metric import format_query_result_dataframe

df_ground_truth, df_predicted = format_query_result_dataframe(
                    ground_truth_query=ground_truth_query,
                    ground_truth_endpoint=ground_truth_endpoint,
                    predicted_query=predicted_query,
                    predicted_endpoint=predicted_endpoint
                )

  Using cached result for query


In [32]:
from experiments.utilities.result_metric import calculate_column_metrics

if df_ground_truth.empty:
    print("Ground truth query result is empty")
else: 
    print("Ground truth query result is not empty. Proceed to calculation")

    if df_predicted.empty:
        print("Predicted query result is empty")
    else: 
        print("Predicted query result is not empty. Proceed to calculation")
        metrics = calculate_column_metrics(df_ground_truth, df_predicted)
        print("Metrics:", metrics)

Ground truth query result is not empty. Proceed to calculation
Predicted query result is not empty. Proceed to calculation
Metrics: {'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


In [ ]:
from typing import Dict

sparql_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX schema: <https://schema.org/>
PREFIX spex: <https://purl.expasy.org/sparql-examples/ontology#>

SELECT ?s ?comment ?select ?target ?federatesWith
WHERE {
    ?s rdfs:comment ?comment ;
       sh:select ?select ;
       schema:target ?target ;
       spex:federatesWith ?federatesWith .
}
"""


def get_sparql_question_meta(ttl_content: str) -> Dict[str, str]:
    g = rdflib.Graph()
    g.parse(data=ttl_content, format='turtle')
    results = g.query(sparql_query)

    print()

    for row in results:
        return {
            "resource": str(row.s),
            "natural_language_question": str(row.comment),
            "query": str(row.select),
            "target_endpoint": str(row.target),
            "federates_with": str(row.federatesWith),
        }

In [1]:
import rdflib
from typing import Dict, Any, List

sparql_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX schema: <https://schema.org/>
PREFIX spex: <https://purl.expasy.org/sparql-examples/ontology#>

SELECT ?s ?comment ?select ?target
WHERE {
    ?s rdfs:comment ?comment ;
       sh:select ?select ;
       schema:target ?target .
}
"""

def get_sparql_question_meta(ttl_content: str) -> Dict[str, Any]:
    g = rdflib.Graph()
    g.parse(data=ttl_content, format='turtle')
    results = g.query(sparql_query)
    
    # Define the spex namespace for federatesWith
    spex = rdflib.Namespace("https://purl.expasy.org/sparql-examples/ontology#")
    
    for row in results:
        # Get all federatesWith values for this subject
        federates_with_list = [str(o) for o in g.objects(row.s, spex.federatesWith)]
        
        return {
            "resource": str(row.s),
            "natural_language_question": str(row.comment),
            "query": str(row.select),
            "target_endpoint": str(row.target),
            "federates_with": federates_with_list,
        }

In [ ]:
import rdflib
import sys
import os
#sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

with open(f"/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/federated_sparql_dataset/examples_federated_02.04.2025/SwissLipids/34.ttl", "r", encoding="utf-8") as f:
    content = f.read()


result = get_sparql_question_meta(content)

print(result)


{'resource': 'https://sparql.swisslipids.org/.well-known/sparql-examples/34', 'natural_language_question': 'Lipids affected by human enzymes', 'query': 'PREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT ?swisslipid  {\n  ?swisslipid owl:equivalentClass ?chebi .\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?rhea rh:side/rh:contains/rh:compound ?compound .\n\t?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?metabolite . \n  }\n  SERVICE <https://sparql.uniprot.org/sparql> {\n    ?catalyticActivityAnnotation up:catalyticActivity/up:catalyzedReaction ?rhea .\n    ?protein up:annotation ?catalyticActivityAnnotation ;\n             up:organism taxon:9606 .\n  }\n}', 'target_endpoint': 'https://sparql.swisslipids.org/sparql/', 'federates_with': ['https://sparql.uniprot.org/sparql', 'https://sparql.rhea-db.org/sparql']}

In [1]:
import sys
import os

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.query_cache import cache_dataset_queries   


result = cache_dataset_queries(
    endpoint_files_map={
        "UniProt": [],
        "Rhea": [],
        "SwissLipids": []
    },
    timeout = 1800,
    dataset_dir = "/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/federated_sparql_dataset/examples_federated_19.04.2025",
    cache_dir = "/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/federated_sparql_dataset/query_cache"
)

Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/118_biosodafrontend_rat_brain_human_cancer
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/51
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/40
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/49
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/53
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/52
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/54
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/43
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examp

In [2]:
print(result)

[]


In [1]:
import sys
import os

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.query_cache import cache_dataset_queries  
from experiments.utilities.query_cache import list_cached_queries



cached_queries = list_cached_queries(cache_dir="/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/federated_sparql_dataset/examples_federated_02.04.2025_queries_with_errors/query_cache")


Expecting value: line 5 column 13 (char 1024)
Expecting value: line 5 column 13 (char 900)
Expecting value: line 5 column 13 (char 783)
Expecting value: line 5 column 13 (char 1201)
Expecting value: line 5 column 13 (char 1012)
Expecting value: line 5 column 13 (char 1005)
Expecting value: line 5 column 13 (char 1180)
Expecting value: line 5 column 13 (char 538)


In [2]:
print(cached_queries)

[]


In [5]:
import sys
import os
import json

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')
from experiments.utilities.query_cache import generate_cache_key, get_cache_path

def get_cache_id_and_result_length(query, endpoint_url, cache_dir=None, verbose=True):
    """
    Returns the cache id and the number of rows in the cached result for a given query and endpoint.
    
    Args:
        query (str): The SPARQL query string.
        endpoint_url (str): The endpoint URL.
        cache_dir (str, optional): The cache directory. If None, uses the default from your utility.
        verbose (bool): If True, prints info when cache file is missing or errors occur.
        
    Returns:
        tuple: (cache_id, result_length)
            - cache_id (str): The MD5 hash key for the cache.
            - result_length (int or None): Number of rows in the result, or None if not found.
    """
    cache_id = generate_cache_key(query, endpoint_url)
    cache_path = get_cache_path(cache_id, cache_dir)
    result_length = None
    if os.path.exists(cache_path):
        try:
            with open(cache_path, "r", encoding="utf-8") as f:
                cache_entry = json.load(f)
            result = cache_entry.get("result", {})
            bindings = result.get("results", {}).get("bindings", [])
            result_length = len(bindings)
        except Exception as e:
            if verbose:
                print(f"Error reading cache file {cache_path}: {e}")
    else:
        if verbose:
            print(f"Cache file does not exist: {cache_path}")
    return cache_id, result_length

# Example usage:
# cache_id, length = get_cache_id_and_result_length(query, endpoint_url)
# print("Cache ID:", cache_id)
# print("Result length:", length)

In [6]:
query = """PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rh: <http://rdf.rhea-db.org/>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>

SELECT DISTINCT ?protein ?chemblEntry
WHERE {
  SERVICE <https://sparql.rhea-db.org/sparql> {
    ?rhea rdfs:subClassOf rh:Reaction ;
      rh:side/rh:contains/rh:compound/rh:chebi/rdfs:subClassOf+ CHEBI:26739 .
  }
  ?ca up:catalyzedReaction ?rhea .
  ?protein up:annotation/up:catalyticActivity ?ca ;
    up:organism taxon:9606 ;
    rdfs:seeAlso ?chemblEntry .
  ?chemblEntry up:database <http://purl.uniprot.org/database/ChEMBL> .
}"""

endpoint_url = "https://sparql.uniprot.org/sparql"

In [7]:
cache_id, length = get_cache_id_and_result_length(query, endpoint_url, cache_dir="/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/federated_sparql_dataset/query_cache_backup")
print("Cache ID:", cache_id)
print("Result length:", length)

Cache file does not exist: /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/federated_sparql_dataset/query_cache_backup/7773596f07263dd96c6bf13110a5377a.json
Cache ID: 7773596f07263dd96c6bf13110a5377a
Result length: None


In [2]:
import os
import json

cache_dir = "/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/federated_sparql_dataset/query_cache"

max_len = -1
max_file = None
max_query = None
max_endpoint = None

for filename in os.listdir(cache_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(cache_dir, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                cache_entry = json.load(f)
            result = cache_entry.get("result", {})
            bindings = result.get("results", {}).get("bindings", [])
            length = len(bindings)
            if length > max_len:
                max_len = length
                max_file = filename
                max_query = cache_entry.get("query")
                max_endpoint = cache_entry.get("endpoint_url")
        except Exception as e:
            print(f"Error reading {filename}: {e}")

print(f"Largest result set: {max_len} rows")
print(f"File: {max_file}")
print(f"Query: {max_query}")
print(f"Endpoint: {max_endpoint}")

Largest result set: 985915 rows
File: d3430bf221c4d0e05d1152ade05b9251.json
Query: PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sachem: <http://bioinfo.uochb.cas.cz/rdf/v1.0/sachem#>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT 
  ?protein
  ?mnemonic
  ?proteinName
  ?ligandSimilarityScore
  ?ligand
WHERE {
  SERVICE <https://idsm.elixir-czech.cz/sparql/endpoint/chebi> {
    ?ssc sachem:compound ?ligand; 
      sachem:score ?ligandSimilarityScore ;
      sachem:similaritySearch ?sss .
        # Smiles of Heme
    ?sss    sachem:query "CC1=C(CCC([O-])=O)C2=[N+]3C1=Cc1c(C)c(C=C)c4C=C5C(C)=C(C=C)C6=[N+]5[Fe-]3(n14)n1c(=C6)c(C)c(CCC([O-])=O)c1=C2";
      sachem:cutoff "8e-1"^^xsd:double ;
      sachem:aromaticityMode sachem:aromaticityDetect ;
      sachem:similarityRadius 1 ;
      sachem:tautomerMode sachem:ignoreTautomers .
  }
  ?protein up:mnemonic ?mnemonic ;
    up:recommendedName/up:fullName ?proteinName ;
    up

In [ ]:
import sys
import os

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.query_cache import cache_dataset_queries, list_cached_queries  


# List cached queries
cached_queries = list_cached_queries("/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/federated_sparql_dataset/query_cache")
print("Cached queries:")
for query in cached_queries:
    print(query)



In [ ]:
import os
import json

cache_dir = "/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/federated_sparql_dataset/query_cache"

none_result_files = []

for filename in os.listdir(cache_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(cache_dir, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                cache_entry = json.load(f)
            result = cache_entry.get("result", "MISSING")
            if result is None:
                none_result_files.append({
                    "file": filename,
                    "query": cache_entry.get("query"),
                    "endpoint_url": cache_entry.get("endpoint_url")
                })
        except Exception as e:
            print(f"Error reading {filename}: {e}")

if none_result_files:
    print(f"Found {len(none_result_files)} cache files with None as result:")
    for entry in none_result_files:
        print(f"File: {entry['file']}\nQuery: {entry['query']}\nEndpoint: {entry['endpoint_url']}\n")
else:
    print("No cache files with None as result found.")

In [1]:
import pandas as pd
import numpy as np
import sys
import os

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')


from experiments.utilities.result_metric import calculate_column_metrics_with_label_similarity

# Seed for reproducibility
np.random.seed(42)

# Expanded DataFrames with 4 columns each
df_ground_truth = pd.DataFrame({
    "Disease": ["Alzheimer's Disease", "Parkinson's Disease"],
    "Gene": ["APP", "SNCA"],
    "Organism": ["Human", "Human"],
    "Location": ["Brain", "Brain"]
})

df_predicted = pd.DataFrame({
    "disease": ["Alzheimer's Disease", "Parkinson's Disease"],
    "gene_symbol": ["APP", "SNCA"],  # purposely slightly different label
    "species": ["Human", "Human"],   # purposely different label
    "location": ["Brain", "Brain"]
})

# Randomize column order
gt_cols = list(df_ground_truth.columns)
pred_cols = list(df_predicted.columns)
np.random.shuffle(gt_cols)
np.random.shuffle(pred_cols)

df_ground_truth = df_ground_truth[gt_cols]
df_predicted = df_predicted[pred_cols]

print("Ground truth columns (random order):", df_ground_truth.columns.tolist())
print("Predicted columns (random order):", df_predicted.columns.tolist())

file_path = "test_file.json"

metrics = calculate_column_metrics_with_label_similarity(
    file_path,
    df_ground_truth,
    df_predicted,
    similarity_threshold=0.5  # Lowered for demo purposes
)

print("Column-wise metrics with label similarity:")
print(metrics)

Ground truth columns (random order): ['Gene', 'Location', 'Disease', 'Organism']
Predicted columns (random order): ['gene_symbol', 'location', 'disease', 'species']
[calculate_column_metrics_with_label_similarity] Calculating metrics for file: test_file.json
exact pairs: 2
remaining gt: 2
Gene -> gene_symbol (similarity: 0.7650271654129028)
Organism -> species (similarity: 0.768046498298645)
gt_tuples: [('Brain', "Alzheimer's Disease", 'APP', 'Human'), ('Brain', "Parkinson's Disease", 'SNCA', 'Human')]
pred_tuples: [('Brain', "Alzheimer's Disease", 'APP', 'Human'), ('Brain', "Parkinson's Disease", 'SNCA', 'Human')]
Column-wise metrics with label similarity:
{'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0, 'predicted_query_result_is_empty': False, 'ground_truth_query_result_is_empty': False}


In [2]:
df_ground_truth.head()

,Gene,Location,Disease,Organism
0,APP,Brain,Alzheimer's Disease,Human
1,SNCA,Brain,Parkinson's Disease,Human


In [3]:
df_predicted.head()

,gene_symbol,location,disease,species
0,APP,Brain,Alzheimer's Disease,Human
1,SNCA,Brain,Parkinson's Disease,Human


In [1]:
import asyncio
import sys
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.evaluation import AgentEvaluator


evaluator = AgentEvaluator(
    endpoint_sets={"SwissLipids": ["34.ttl"]},
    test=True,
    experiment_dir="eval",
    timeout=300,
    project_name_langsmith="test",
    tracked_token_nodes= ["question_understanding", "planning"],
    dataset_dir="/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/federated_sparql_dataset/examples_federated_19.04.2025"
)

await evaluator.run_all_tests()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sebastian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Processing endpoint set: SwissLipids
Processed: 34.ttl
Processed 1 files from SwissLipids
Saved metadata for 1 files from 1 endpoint sets to:
  - /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-05-04_22-30-12/testset_meta_data.json
Processing item: SwissLipids/34.ttl
Sending question 1/1 to agent...


I0000 00:00:1746390614.773312 2388175 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Got result for question 1/1
Querying ground truth endpoint...
  Using cached result for query
Querying predicted endpoint...
SP-BLEU 0.25306253194973727
METEOR 0.5174784057934232
F1-score:  0
SP-F1-score:  0
Number of invalid queries(None): 0
Saved query comparison to: /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-05-04_22-30-12/34_comparison.ttl


'/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-05-04_22-30-12/evaluation_dataset.json'

In [1]:
import asyncio
import sys
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from entity_indexing.endpoint_loader import query_sparql_wrapper


test_query = """
PREFIX rh: <http://rdf.rhea-db.org/>


ASK WHERE {SERVICE <https://sparql.rhea-db.org/sparql> {
    ?rhea rh:side/rh:contains/rh:compound ?compound .
	?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?metabolite . 
  }
}"""

test_query_wrong = """PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX rh:    <http://rdf.rhea-db.org/>

ASK WHERE {
  SERVICE <https://sparql.rhea-db.org/sparql> {
    # Using a made-up CHEBI ID that has no corresponding compound
    VALUES (?chebi) I:0000000) }

    # Any reaction side containing a compound…
    ?rhea rh:side/rh:contains/rh:compound ?compound .

    # …where that compound has the bogus CHEBI ID
    ?compound rh:chebi ?chebi .
  }
}
"""


message=query_sparql_wrapper(test_query_wrong, "https://sparql.uniprot.org/sparql")

print(message)

    



QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'<!DOCTYPE html SYSTEM "about:legacy-compat">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en"><head><title>UniProt</title><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"/><link href="/" rel="home"/><link href="/base.css" type="text/css" rel="stylesheet"/><link type="image/vnd.microsoft.icon" href="/uniprot-favicon.ico" rel="shortcut icon"/><link href="/uniprot-sparql.css" type="text/css" rel="stylesheet"/><link href="http://creativecommons.org/licenses/by/4.0/" rel="license"/><script src="https://cdn.jsdelivr.net/npm/yasgui-yasqe@2.11.22/dist/yasqe.bundled.min.js"></script><link type="text/css" rel="stylesheet" href="https://cdn.jsdelivr.net/yasqe/2.2/yasqe.min.css"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="no-referrer" name="referrer"/></head><body typeof="schema:WebPage"><nav id="nav-head